In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [2]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
%matplotlib inline 
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.tree import export_graphviz
import matplotlib.pyplot as plt

import lightgbm as lgbm
import gc
import xgboost as xgb
import pickle as pickle


from catboost import CatBoostRegressor


In [24]:
items           = pd.read_csv('items.csv',usecols=["item_id", "item_category_id"])
item_categories = pd.read_csv('item_categories.csv')
shops           = pd.read_csv('shops.csv')
sales_train     = pd.read_csv('sales_train.csv.gz')
test            = pd.read_csv('test.csv.gz')

In [4]:
items['item_id'] = pd.to_numeric(items['item_id'],downcast='unsigned')
items['item_category_id'] = pd.to_numeric(items['item_category_id'],downcast='unsigned')

In [5]:
sales_train['date'] = sales_train['date'].astype('category')
sales_train['date_block_num'] = pd.to_numeric(sales_train['date_block_num'],downcast='unsigned')
sales_train['shop_id'] = pd.to_numeric(sales_train['shop_id'],downcast='unsigned')
sales_train['item_price'] = sales_train['item_price'].astype('int')
sales_train['item_price'] = pd.to_numeric(sales_train['item_price'],downcast='unsigned')
sales_train['item_cnt_day'] = pd.to_numeric(sales_train['item_cnt_day'],downcast='signed')


In [6]:
transactions = sales_train
transactions[['day','month', 'year']] = transactions['date'].str.split('.', expand=True).astype(int)
transactions['day'] = pd.to_numeric(transactions['day'],downcast='unsigned')
transactions['month'] = pd.to_numeric(transactions['month'],downcast='unsigned')
transactions['year'] = pd.to_numeric(transactions['year'],downcast='unsigned')


In [7]:
transactions = transactions.set_index('item_id').join(items.set_index('item_id'))
transactions.reset_index(inplace=True)
transactions = transactions[transactions['year'] != 2013]
transactions['y'] = pd.to_numeric(transactions.groupby(['date_block_num', 'item_id', 'shop_id'])['item_cnt_day'].transform('sum')\
                .clip(0,20), downcast='unsigned')

In [8]:
len(transactions)

1668287

In [9]:
transactions['turnover'] = transactions['item_price'] * transactions['item_cnt_day']
transactions['turnover'] = pd.to_numeric(transactions['turnover'], downcast='unsigned')

In [10]:
transactions['item_first_block'] = pd.to_numeric(transactions.groupby('item_id')['date_block_num'].transform(np.min), downcast='unsigned')
transactions['item_last_block'] = pd.to_numeric(transactions.groupby('item_id')['date_block_num'].transform(np.max), downcast='unsigned')

transactions['is_first_two_blocks'] = \
                    transactions['date_block_num'].isin([transactions['item_first_block']+1,transactions['item_first_block']+2])


transactions['is_last_two_blocks'] = \
                transactions['date_block_num'].isin([transactions['item_last_block']-1,transactions['item_last_block']])
                                

In [11]:
number_of_items = transactions['item_id'].nunique()
print("number_of_items:", number_of_items)
number_of_categories = transactions['item_category_id'].nunique()
print("number_of_categories:", number_of_categories)
number_of_shops = transactions['shop_id'].nunique()
print("number_of_shops:", number_of_shops)
number_of_days = 365 + 365 - 30 - 31
print("number_of_days:", number_of_days)
number_of_blocks = transactions['date_block_num'].nunique()
print("number_of_blocks:", number_of_blocks)
total_sales = transactions['item_cnt_day'].sum()
print("total_sales:", total_sales)
total_turnover = transactions['turnover'].sum()
print("total_turnover:", total_turnover)
average_price = transactions['item_price'].mean()
print("average_price:", average_price)

number_of_items: 17054
number_of_categories: 79
number_of_shops: 55
number_of_days: 669
number_of_blocks: 22
total_sales: 2085473
total_turnover: 2181307117
average_price: 1015.4701882829513


#ITEM

-UNITS
item_units
item_block_units
item_mean_units_block
item_day_units
item_mean_units_day
item_max_units_block
item_min_units_block
item_max_units_day
item_min_units_day

-TURNOVER
item_turnover
item_block_turnover
item_mean_turnover_block
item_day_turnover
item_mean_turnover_day
item_max_turnover_block
item_min_turnover_block
item_max_turnover_day
item_min_turnover_day


-TIME
item_days_of_activity
item_blocks_of_activity
item_mean_day_between_activity
item_longest_stretch_days_without_activity
item_longest_stretch_blocks_without_activity
item_longest_stretch_block_with_activity
item_number_of_consecutive_days_with_activity
item_days_between_start_and_first_activity
item_blocks_between_start_and_first_activity
item_first_block
item_last_block
item_first_day
item_last_day
item_activity_on_all_blocks


-PRICE
item_mean_price
item_mean_price_block
item_min_price
item_max_price
item_number_different_prices
item_price_amplitude (%age min/max)
item_deviation_mean_category_price


-TREND
is_first_two_full_blocks (actually second/third to make sure we have a "full" block if this was a new release !!!!
is_last_two_blocks
item_first_two_blocks_units
item_last_two_blocks_units
item_fluctuation_units_first_last_blocks
item_first_two_blocks_mean_price
item_last_two_blocks_mean_price
item_fluctuation_price_first_last_blocks

-ENCODINGS
item_share_of_total_units
item_share_of_total_gross
item_share_of_category_units
item_share_of_category_turnover

In [15]:
gc.collect()
transactions_items = transactions.copy()
transactions_items_blocks = transactions.copy()

In [16]:
transactions_items_blocks['item_block_units'] = pd.to_numeric(transactions_items_blocks.groupby(['item_id','date_block_num'])['item_cnt_day'].transform(np.sum), downcast='unsigned')    
transactions_items_blocks['item_block_turnover'] = pd.to_numeric(transactions_items_blocks.groupby(['item_id','date_block_num'])['turnover'].transform(np.sum), downcast='unsigned')    
transactions_items_blocks['item_mean_price_block'] = pd.to_numeric(transactions_items_blocks.groupby(['item_id', 'date_block_num'])['item_price'].transform(np.mean), downcast='float')    

In [17]:
transactions_items['item_units'] = pd.to_numeric(transactions_items.groupby(['item_id'])['item_cnt_day'].transform(np.sum), downcast='unsigned') 
transactions_items['item_mean_units_block'] = pd.to_numeric(transactions_items_blocks.groupby(['item_id'])['item_block_units'].transform(np.mean), downcast='float') 
transactions_items['item_day_units'] = pd.to_numeric(transactions_items.groupby(['item_id','date'])['item_cnt_day'].transform(np.sum), downcast='unsigned') 
transactions_items['item_mean_units_day'] = pd.to_numeric(transactions_items.groupby(['item_id'])['item_day_units'].transform(np.mean), downcast='float') 
transactions_items['item_max_units_block'] = pd.to_numeric(transactions_items_blocks.groupby(['item_id'])['item_block_units'].transform(np.max), downcast='unsigned') 
transactions_items['item_min_units_block'] = pd.to_numeric(transactions_items_blocks.groupby(['item_id'])['item_block_units'].transform(np.min), downcast='unsigned') 
transactions_items['item_max_units_day'] = pd.to_numeric(transactions_items.groupby(['item_id'])['item_day_units'].transform(np.max), downcast='unsigned') 
transactions_items['item_min_units_day'] = pd.to_numeric(transactions_items.groupby(['item_id'])['item_day_units'].transform(np.min), downcast='unsigned') 

In [18]:
transactions_items['item_turnover'] = pd.to_numeric(transactions_items.groupby(['item_id'])['turnover'].transform(np.sum), downcast='unsigned') 
transactions_items['item_mean_turnover_block'] = pd.to_numeric(transactions_items_blocks.groupby(['item_id'])['item_block_turnover'].transform(np.mean), downcast='float') 
transactions_items['item_day_turnover'] = pd.to_numeric(transactions_items.groupby(['item_id','date'])['turnover'].transform(np.sum), downcast='unsigned') 
transactions_items['item_mean_turnover_day'] = pd.to_numeric(transactions_items.groupby(['item_id'])['turnover'].transform(np.mean), downcast='float') 
transactions_items['item_max_turnover_block'] = pd.to_numeric(transactions_items_blocks.groupby(['item_id'])['item_block_turnover'].transform(np.max), downcast='unsigned') 
transactions_items['item_min_turnover_block'] = pd.to_numeric(transactions_items_blocks.groupby(['item_id'])['item_block_turnover'].transform(np.min), downcast='unsigned') 
transactions_items['item_max_turnover_day'] = pd.to_numeric(transactions_items.groupby(['item_id'])['item_day_turnover'].transform(np.max), downcast='unsigned') 
transactions_items['item_min_turnover_day'] = pd.to_numeric(transactions_items.groupby(['item_id'])['item_day_turnover'].transform(np.min), downcast='unsigned') 

In [19]:
transactions_items['item_days_of_activity'] = pd.to_numeric(transactions_items.groupby(['item_id'])['date'].transform("nunique"), downcast='unsigned') 
transactions_items['item_blocks_of_activity'] = pd.to_numeric(transactions_items.groupby(['item_id'])['date_block_num'].transform("nunique"), downcast='unsigned') 

def get_number_of_days_since_start(day,month, year):
    days = 0
    if year == 2015:
        days = 365
    def is_even(num):
        return num % 2 == 0
    half_of_month = int(month/2)
    even = (30*half_of_month) + (31*half_of_month)
    if is_even(month):
        days = days + even - 30 - day
    else:
        days = days + even + day
    return days

transactions_items['item_days_since_start'] = pd.to_numeric(transactions_items.apply(lambda row: get_number_of_days_since_start(row['day'],row['month'], row['year']),axis=1), downcast='unsigned') 

def get_average_days_between_sales(days):
    days = sorted(np.unique(days))
    if len(days) == 0:
        return 9999
    if len(days) == 1:
        return 999
    return np.mean(np.ediff1d(days)) / len(days)

average_days_between_sales = transactions_items.groupby(['item_id'])['item_days_since_start'].apply(list).apply(lambda x: get_average_days_between_sales(x))

transactions_items['item_mean_day_between_activity'] = pd.to_numeric(transactions_items['item_id'].map(average_days_between_sales), downcast='unsigned') 


def get_max_stretch_without_sales_days(days):
    days = np.unique(days)
    max_stretch = 0
    len_days = len(days)
    for index,day in enumerate(sorted(days)):
        if index == len_days - 1:
            return max_stretch
        next_day = days[index+1]
        stretch = next_day - day
        if stretch > max_stretch:
            max_stretch = stretch
            

        
max_stretch_without_sales_day = transactions_items.groupby(['item_id'])['item_days_since_start'].apply(list).apply(lambda x: get_max_stretch_without_sales_days(x))

transactions_items['item_longest_stretch_days_without_activity'] = pd.to_numeric(transactions_items['item_id'].map(max_stretch_without_sales_day), downcast='unsigned') 

In [20]:
gc.collect()

def get_max_stretch_without_sales_block(blocks):
    blocks = np.unique(blocks)
    max_stretch = 0
    len_blocks = len(blocks)
    for index,block in enumerate(sorted(blocks)):
        if index == len_blocks - 1:
            return max_stretch
        next_block = blocks[index+1]
        stretch = next_block - block
        if stretch > max_stretch:
            max_stretch = stretch
            

        
item_longest_stretch_blocks_without_activity = transactions_items.groupby(['item_id'])['date_block_num']\
                                    .apply(list).apply(lambda x: get_max_stretch_without_sales_block(x))

transactions_items['item_longest_stretch_blocks_without_activity'] = pd.to_numeric(transactions_items['item_id'].map(item_longest_stretch_blocks_without_activity), downcast='unsigned') 



def get_longest_stretch(following_pairs, n=1,new_n=1):
    #print("following_pairs", following_pairs, " n: ", n, " new_n: ", new_n)
    len_pairs = len(following_pairs)
    if len_pairs == 0:
        return 0
    if len_pairs == 1:
        if new_n > n:
            return new_n
        return n
    if following_pairs[1][0] == following_pairs[0][1]:
        new_n+=1
    else:
        if new_n > n:
            n=new_n
        new_n=1
    return get_longest_stretch(following_pairs[1:], n,new_n)


assert(get_longest_stretch([]) == 0)
assert(get_longest_stretch([[1, 2], [2,3], [3, 4], [4,5] ,[8,9], [11, 12]]) == 4)
assert(get_longest_stretch([[-1, 0],[1, 2], [2,3], [3, 4], [4,5] ,[8,9], [11, 12]]) == 4)
assert(get_longest_stretch([[1, 2], [4,5] ,[8,9], [9,10],[10, 11]]) == 3)
assert(get_longest_stretch([[1, 2], [4,5] ,[8,9], [9,10],[10, 11],[20, 21], [25,26]]) == 3)
assert(get_longest_stretch([[1, 2], [4,5] ,[8,9], [9,10],[10, 11],[14, 15], [15,16]]) == 3)
assert(get_longest_stretch([[1, 2], [4,5] ,[8,9], [9,10],[10, 11],[14, 15], [15,16],[18,19] ,[22,23], [23,24],[24, 25]]) == 3)
assert(get_longest_stretch([[1, 2], [4,5], [7, 8]]) == 1)
assert(get_longest_stretch([[1, 2], [5, 6], [6, 7], [7,8], [14, 15]]) == 3)
assert(get_longest_stretch([[1, 2], [5, 6], [6, 7], [7,8], [14, 15], [15, 16]]) == 3)
assert(get_longest_stretch([[1, 2], [5, 6], [6, 7], [7,8], [14, 15], [15, 16], [16, 17]]) == 3)
assert(get_longest_stretch([[1, 2], [5, 6], [6, 7], [14, 15], [15, 16], [16, 17]]) == 3)
assert(get_longest_stretch([[1, 2], [5, 6], [6, 7], [7,8], [14, 15], [20, 21], [21, 22], [22,23],[23,24]]) == 4)


def get_following_pairs(pairs):
    pairs = np.unique(pairs)
    len_pairs = len(pairs)
    following = []
    for index,pair in enumerate(sorted(pairs)):
        if index == len_pairs - 1:
            return following
        next_pair = pairs[index+1]
        if next_pair == pair + 1:
            following.append([pair, next_pair])
        
assert(get_following_pairs([1,2,5,6,7,8,9,11,12,15]) == [[1, 2], [5, 6], [6, 7], [7, 8], [8, 9], [11, 12]])
assert(get_following_pairs([1,2,5,6,7,10]) == [[1, 2], [5, 6], [6, 7]])
assert(get_following_pairs([1,2,4,5,7,9,10]) == [[1, 2], [4, 5], [9,10]])
assert(get_following_pairs([1,2,4,5,7,9,10,11,12,15]) == [[1, 2], [4, 5], [9,10],[10,11],[11,12]])


item_longest_stretch_block_with_activity = transactions_items.groupby(['item_id'])['date_block_num']\
                                    .apply(list).apply(lambda x: get_longest_stretch(get_following_pairs(x)))

transactions_items['item_longest_stretch_block_with_activity'] = pd.to_numeric(transactions_items['item_id'].map(item_longest_stretch_block_with_activity), downcast='unsigned') 


item_number_of_consecutive_days_with_activity = transactions_items.groupby(['item_id'])['item_days_since_start']\
                                    .apply(list).apply(lambda x: len(get_following_pairs(x)))
    
transactions_items['item_number_of_consecutive_days_with_activity'] = pd.to_numeric(transactions_items['item_id'].map(item_number_of_consecutive_days_with_activity), downcast='unsigned') 

In [21]:
def get_units_between_first_and_last(units):
    return np.max(units) - np.min(units)

item_days_between_start_and_first_activity = transactions_items.groupby(['item_id'])['item_days_since_start'].apply(list).apply(lambda x: get_units_between_first_and_last(x))
transactions_items['item_days_between_start_and_first_activity'] = pd.to_numeric(transactions_items['item_id'].map(item_days_between_start_and_first_activity), downcast='unsigned') 

item_blocks_between_start_and_first_activity = transactions_items.groupby(['item_id'])['date_block_num'].apply(list).apply(lambda x: get_units_between_first_and_last(x))
transactions_items['item_blocks_between_start_and_first_activity'] = pd.to_numeric(transactions_items['item_id'].map(item_blocks_between_start_and_first_activity), downcast='unsigned') 

In [22]:

transactions_items['item_first_day'] = pd.to_numeric(transactions_items.groupby('item_id')['item_days_since_start'].transform(np.min), downcast='unsigned') 
transactions_items['item_last_day'] = pd.to_numeric(transactions_items.groupby('item_id')['item_days_since_start'].transform(np.max), downcast='unsigned') 

item_activity_on_all_blocks = transactions_items.groupby('item_id')['date_block_num'].nunique().apply(lambda x: x==number_of_blocks)
transactions_items['item_activity_on_all_blocks'] = transactions_items['item_id'].map(item_activity_on_all_blocks)

In [23]:
transactions_items['item_mean_price'] = pd.to_numeric(transactions_items.groupby('item_id')['item_price'].transform(np.mean), downcast='float') 
transactions_items['item_min_price'] = pd.to_numeric(transactions_items.groupby('item_id')['item_price'].transform(np.min), downcast='unsigned') 
transactions_items['item_max_price'] = pd.to_numeric(transactions_items.groupby('item_id')['item_price'].transform(np.max), downcast='unsigned') 
transactions_items['item_number_different_prices'] = pd.to_numeric(transactions_items.groupby('item_id')['item_price'].transform('nunique'), downcast='unsigned') 
transactions_items['item_price_amplitude'] = pd.to_numeric(((transactions_items['item_max_price'] - transactions_items['item_min_price'] ) / transactions_items['item_min_price']) * 100, downcast='float') 
transactions_items['category_mean_price'] = pd.to_numeric(transactions_items.groupby('item_category_id')['item_price'].transform(np.mean), downcast='float') 
transactions_items['item_deviation_mean_category_price'] =  pd.to_numeric(((transactions_items['item_mean_price'] - transactions_items['category_mean_price'] ) / transactions_items['category_mean_price']) * 100, downcast='float') 

In [24]:
item_first_two_blocks_units = transactions_items.groupby(['item_id','is_first_two_blocks'], as_index=False)['item_cnt_day'].sum()
item_first_two_blocks_units = item_first_two_blocks_units[item_first_two_blocks_units['is_first_two_blocks'] == True].set_index('item_id').iloc[:,1]
transactions_items['item_first_two_blocks_units'] = pd.to_numeric(transactions_items['item_id'].map(item_first_two_blocks_units), downcast='unsigned') 

item_last_two_blocks_units = transactions_items.groupby(['item_id','is_last_two_blocks'], as_index=False)['item_cnt_day'].sum()
item_last_two_blocks_units = item_last_two_blocks_units[item_last_two_blocks_units['is_last_two_blocks'] == True].set_index('item_id').iloc[:,1]
transactions_items['item_last_two_blocks_units'] = pd.to_numeric(transactions_items['item_id'].map(item_last_two_blocks_units), downcast='unsigned') 

transactions_items['item_fluctuation_units_first_last_blocks'] =  pd.to_numeric(((transactions_items['item_first_two_blocks_units'] - transactions_items['item_last_two_blocks_units'] ) / \
                                                             transactions_items['item_first_two_blocks_units']) * 100 * -1, downcast='float') 


item_first_two_blocks_mean_price = transactions_items.groupby(['item_id','is_first_two_blocks'], as_index=False)['item_price'].mean()
item_first_two_blocks_mean_price = item_first_two_blocks_mean_price[item_first_two_blocks_mean_price['is_first_two_blocks'] == True].set_index('item_id').iloc[:,1]
transactions_items['item_first_two_blocks_mean_price'] = pd.to_numeric(transactions_items['item_id'].map(item_first_two_blocks_mean_price), downcast='unsigned') 

item_last_two_blocks_mean_price = transactions_items.groupby(['item_id','is_last_two_blocks'], as_index=False)['item_price'].mean()
item_last_two_blocks_mean_price = item_last_two_blocks_mean_price[item_last_two_blocks_mean_price['is_last_two_blocks'] == True].set_index('item_id').iloc[:,1]
transactions_items['item_last_two_blocks_mean_price'] = pd.to_numeric(transactions_items['item_id'].map(item_last_two_blocks_mean_price), downcast='unsigned') 

transactions_items['item_fluctuation_price_first_last_blocks'] =  pd.to_numeric(((transactions_items['item_first_two_blocks_mean_price'] - transactions_items['item_last_two_blocks_mean_price'] ) / \
                                                             transactions_items['item_first_two_blocks_mean_price'])  * 100 * -1, downcast='float') 

In [25]:
transactions_items['item_share_of_total_units'] = pd.to_numeric(transactions_items['item_units'] * 100 / total_sales , downcast='float') 

transactions_items['item_share_of_total_turnover'] = pd.to_numeric(transactions_items['item_turnover'] * 100 / total_turnover, downcast='float') 

transactions_items['category_units'] = pd.to_numeric(transactions_items.groupby('item_category_id')['item_cnt_day'].transform(np.sum), downcast='unsigned') 
transactions_items['item_share_of_category_units'] = pd.to_numeric(transactions_items['item_units'] * 100 / transactions_items['category_units'], downcast='float') 

transactions_items['category_turnover'] = pd.to_numeric(transactions_items.groupby('item_category_id')['turnover'].transform(np.sum), downcast='unsigned') 
transactions_items['item_share_of_category_turnover'] = pd.to_numeric(transactions_items['item_turnover'] * 100 / transactions_items['category_turnover'], downcast='float') 


In [27]:
transactions_items.to_pickle("pickled/transactions_items")
transactions_items_blocks.to_pickle("pickled/transactions_items_blocks")

del transactions_items
del transactions_items_blocks
gc.collect()

114

#CATEGORY

-UNITS
category_units
category_block_units
category_mean_units_block
category_day_units
category_mean_units_day
category_max_units_block
category_min_units_block
category_max_units_day
category_min_units_day

-TURNOVER
category_turnover
category_block_turnover
category_mean_turnover_block
category_day_turnover
category_mean_turnover_day
category_max_turnover_block
category_min_turnover_block
category_max_turnover_day
category_min_turnover_day


-PRICE
category_mean_price
category_mean_price_block
category_min_price
category_max_price


-TREND
category_first_two_blocks_units
category_last_two_blocks_units
category_fluctuation_units_first_last_blocks
category_first_two_blocks_mean_price
category_last_two_blocks_mean_price
category_fluctuation_price_first_last_blocks

-SUBCATEGORY
subcategory
subcategory 1hot

-UNITS
subcategory_units
subcategory_block_units
subcategory_mean_units_block
subcategory_day_units
subcategory_mean_units_day
subcategory_max_units_block
subcategory_min_units_block
subcategory_max_units_day
subcategory_min_units_day

-TURNOVER
subcategory_turnover
subcategory_block_turnover
subcategory_mean_turnover_block
subcategory_day_turnover
subcategory_mean_turnover_day
subcategory_max_turnover_block
subcategory_min_turnover_block
subcategory_max_turnover_day
subcategory_min_turnover_day

-ENCODINGS
category_share_of_total_units
category_share_of_total_gross
subcategory_share_of_total_units
subcategory_share_of_total_gross

-TREND
subcategory_first_two_blocks_units
subcategory_last_two_blocks_units
subcategory_fluctuation_units_first_last_blocks
subcategory_first_two_blocks_mean_price
subcategory_last_two_blocks_mean_price
subcategory_fluctuation_price_first_last_blocks

In [28]:
gc.collect()
transactions_categories = transactions.copy()
transactions_categories_blocks = transactions.copy()

In [29]:
sub_cats = {}
for i in range(1,8):
    sub_cats[i] = "Accessories"
sub_cats[8] = "Tickets"
sub_cats[9] = "Delivery of goods"
for i in range(10,18):
    sub_cats[i] = "Consoles"
for i in range(18,25):
    sub_cats[i] = "Game for Consoles"
sub_cats[25] = "Accessories for Games"
sub_cats[26] = "Android Games"
sub_cats[27] = "MAC Games"
for i in range(28,32):
    sub_cats[i] = "PC Games"
for i in range(32,37):
    sub_cats[i] = "Payment Cards"
for i in range(37,40):
    sub_cats[i] = "Cinema - Blu-ray"
sub_cats[40] = "Cinema - DVD"
sub_cats[41] = "Cinema - Collectible"
for i in range(42,46):
    sub_cats[i] = "Audiobooks"
for i in range(46,55):
    sub_cats[i] = "Books"
for i in range(55,57):
    sub_cats[i] = "Music - CD"
sub_cats[57] = "Music - MP3"
sub_cats[58] = "Music - Vinyl"
sub_cats[59] = "Music - Music Video"
sub_cats[60] = "Music - Gift Edition"
for i in range(61,74):
    sub_cats[i] = "Gifts"
for i in range(73,79):
    sub_cats[i] = "Software"
sub_cats[79] = "Utility"
for i in range(80,84):
    sub_cats[i] = "Misc"
    
    
transactions_categories['subcategory'] = transactions_categories['item_category_id'].apply(lambda x: sub_cats[x]).astype('category')
transactions_categories_blocks['subcategory'] = transactions_categories_blocks['item_category_id'].apply(lambda x: sub_cats[x]).astype('category')

In [30]:
transactions_categories_blocks['category_block_units'] = pd.to_numeric(transactions_categories_blocks.groupby(['item_category_id','date_block_num'])['item_cnt_day'].transform(np.sum), downcast='unsigned') 
transactions_categories_blocks['category_block_turnover'] = pd.to_numeric(transactions_categories_blocks.groupby(['item_category_id','date_block_num'])['turnover'].transform(np.sum), downcast='unsigned') 
transactions_categories_blocks['category_mean_price_block'] = pd.to_numeric(transactions_categories_blocks.groupby(['item_category_id', 'date_block_num'])['item_price'].transform(np.mean), downcast='float') 
transactions_categories_blocks['subcategory_block_units'] = pd.to_numeric(transactions_categories_blocks.groupby(['subcategory','date_block_num'])['item_cnt_day'].transform(np.sum), downcast='unsigned') 
transactions_categories_blocks['subcategory_block_turnover'] = pd.to_numeric(transactions_categories_blocks.groupby(['subcategory','date_block_num'])['turnover'].transform(np.sum), downcast='unsigned') 
transactions_categories_blocks['subcategory_mean_price_block'] = pd.to_numeric(transactions_categories_blocks.groupby(['subcategory', 'date_block_num'])['item_price'].transform(np.mean), downcast='float') 


In [31]:
transactions_categories['category_units'] = pd.to_numeric(transactions_categories.groupby(['item_category_id'])['item_cnt_day'].transform(np.sum), downcast='unsigned') 
transactions_categories['category_mean_units_block'] = pd.to_numeric(transactions_categories_blocks.groupby(['item_category_id'])['category_block_units'].transform(np.mean), downcast='float') 
transactions_categories['category_day_units'] = pd.to_numeric(transactions_categories.groupby(['item_category_id','date'])['item_cnt_day'].transform(np.sum), downcast='unsigned') 
transactions_categories['category_mean_units_day'] = pd.to_numeric(transactions_categories.groupby(['item_category_id'])['category_day_units'].transform(np.mean), downcast='float') 
transactions_categories['category_max_units_block'] = pd.to_numeric(transactions_categories_blocks.groupby(['item_category_id'])['category_block_units'].transform(np.max), downcast='unsigned') 
transactions_categories['category_min_units_block'] = pd.to_numeric(transactions_categories_blocks.groupby(['item_category_id'])['category_block_units'].transform(np.min), downcast='unsigned') 
transactions_categories['category_max_units_day'] = pd.to_numeric(transactions_categories.groupby(['item_category_id'])['category_day_units'].transform(np.max), downcast='unsigned') 
transactions_categories['category_min_units_day'] = pd.to_numeric(transactions_categories.groupby(['item_category_id'])['category_day_units'].transform(np.min), downcast='unsigned') 

In [32]:
transactions_categories['category_turnover'] = pd.to_numeric(transactions_categories.groupby(['item_category_id'])['turnover'].transform(np.sum), downcast='unsigned') 
transactions_categories['category_mean_turnover_block'] = pd.to_numeric(transactions_categories_blocks.groupby(['item_category_id'])['category_block_turnover'].transform(np.mean), downcast='float') 
transactions_categories['category_day_turnover'] = pd.to_numeric(transactions_categories.groupby(['item_category_id','date'])['turnover'].transform(np.sum), downcast='unsigned') 
transactions_categories['category_mean_turnover_day'] = pd.to_numeric(transactions_categories.groupby(['item_category_id'])['category_day_turnover'].transform(np.mean), downcast='float') 
transactions_categories['category_max_turnover_block'] = pd.to_numeric(transactions_categories_blocks.groupby(['item_category_id'])['category_block_turnover'].transform(np.max), downcast='unsigned') 
transactions_categories['category_min_turnover_block'] = pd.to_numeric(transactions_categories_blocks.groupby(['item_category_id'])['category_block_turnover'].transform(np.min), downcast='unsigned') 
transactions_categories['category_max_turnover_day'] = pd.to_numeric(transactions_categories.groupby(['item_category_id'])['category_day_turnover'].transform(np.max), downcast='unsigned') 
transactions_categories['category_min_turnover_day'] = pd.to_numeric(transactions_categories.groupby(['item_category_id'])['category_day_turnover'].transform(np.min), downcast='unsigned') 

In [33]:
transactions_categories['category_mean_price'] = pd.to_numeric(transactions_categories.groupby('item_category_id')['item_price'].transform(np.mean), downcast='float') 
transactions_categories['category_min_price'] = pd.to_numeric(transactions_categories.groupby('item_category_id')['item_price'].transform(np.min), downcast='unsigned')
transactions_categories['category_max_price'] = pd.to_numeric(transactions_categories.groupby('item_category_id')['item_price'].transform(np.max), downcast='unsigned')

In [34]:
category_first_two_blocks_units = transactions_categories.groupby(['item_category_id','is_first_two_blocks'], as_index=False)['item_cnt_day'].sum()
category_first_two_blocks_units = category_first_two_blocks_units[category_first_two_blocks_units['is_first_two_blocks'] == True].set_index('item_category_id').iloc[:,1]
transactions_categories['category_first_two_blocks_units'] = pd.to_numeric(transactions_categories['item_category_id'].map(category_first_two_blocks_units), downcast='unsigned')

category_last_two_blocks_units = transactions_categories.groupby(['item_category_id','is_last_two_blocks'], as_index=False)['item_cnt_day'].sum()
category_last_two_blocks_units = category_last_two_blocks_units[category_last_two_blocks_units['is_last_two_blocks'] == True].set_index('item_category_id').iloc[:,1]
transactions_categories['category_last_two_blocks_units'] = pd.to_numeric(transactions_categories['item_category_id'].map(category_last_two_blocks_units), downcast='unsigned')

transactions_categories['category_fluctuation_units_first_last_blocks'] =  pd.to_numeric(((transactions_categories['category_first_two_blocks_units'] - transactions_categories['category_last_two_blocks_units'] ) / \
                                                             transactions_categories['category_first_two_blocks_units']) * 100 * -1, downcast='float') 


category_first_two_blocks_mean_price = transactions_categories.groupby(['item_category_id','is_first_two_blocks'], as_index=False)['item_price'].mean()
category_first_two_blocks_mean_price = category_first_two_blocks_mean_price[category_first_two_blocks_mean_price['is_first_two_blocks'] == True].set_index('item_category_id').iloc[:,1]
transactions_categories['category_first_two_blocks_mean_price'] = pd.to_numeric(transactions_categories['item_category_id'].map(category_first_two_blocks_mean_price), downcast='unsigned')

category_last_two_blocks_mean_price = transactions_categories.groupby(['item_category_id','is_last_two_blocks'], as_index=False)['item_price'].mean()
category_last_two_blocks_mean_price = category_last_two_blocks_mean_price[category_last_two_blocks_mean_price['is_last_two_blocks'] == True].set_index('item_category_id').iloc[:,1]
transactions_categories['category_last_two_blocks_mean_price'] = pd.to_numeric(transactions_categories['item_category_id'].map(category_last_two_blocks_mean_price), downcast='unsigned')

transactions_categories['category_fluctuation_price_first_last_blocks'] =  pd.to_numeric(((transactions_categories['category_first_two_blocks_mean_price'] - transactions_categories['category_last_two_blocks_mean_price'] ) / \
                                                             transactions_categories['category_first_two_blocks_mean_price'])  * 100 * -1, downcast='float') 

In [35]:
transactions_categories['video_game'] = transactions_categories["item_category_id"].isin(list(range(18,32)))
transactions_categories['gaming_old_gen'] = transactions_categories["item_category_id"].isin([10,11,15,18,19,23])
transactions_categories['gaming_new_gen'] = transactions_categories["item_category_id"].isin([12,14,16,20,22,24])
transactions_categories['pc_games'] = transactions_categories["item_category_id"].isin(list(range(27,32)))
transactions_categories['payment_cards'] = transactions_categories["item_category_id"].isin(list(range(32,37)))
transactions_categories['movies'] = transactions_categories["item_category_id"].isin(list(range(37,42)))
transactions_categories['movies_niche'] = transactions_categories["item_category_id"].isin([38,39])
transactions_categories['books'] = transactions_categories["item_category_id"].isin([42,55])
transactions_categories['music'] = transactions_categories["item_category_id"].isin(list(range(55,61)))
transactions_categories['music_CD'] = transactions_categories["item_category_id"].isin([55,56])
transactions_categories['music_vinyl'] = transactions_categories["item_category_id"].isin([58])
transactions_categories['gifts'] = transactions_categories["item_category_id"].isin(list(range(61,72)))
transactions_categories['software'] = transactions_categories["item_category_id"].isin(list(range(73,79)))

In [36]:
transactions_categories['subcategory_units'] = pd.to_numeric(transactions_categories.groupby(['subcategory'])['item_cnt_day'].transform(np.sum), downcast='unsigned')
transactions_categories['subcategory_mean_units_block'] = pd.to_numeric(transactions_categories_blocks.groupby(['subcategory'])['subcategory_block_units'].transform(np.mean), downcast='float') 
transactions_categories['subcategory_day_units'] = pd.to_numeric(transactions_categories.groupby(['subcategory','date'])['item_cnt_day'].transform(np.sum), downcast='unsigned')
transactions_categories['subcategory_mean_units_day'] = pd.to_numeric(transactions_categories.groupby(['subcategory'])['subcategory_day_units'].transform(np.mean), downcast='float') 
transactions_categories['subcategory_max_units_block'] = pd.to_numeric(transactions_categories_blocks.groupby(['subcategory'])['subcategory_block_units'].transform(np.max), downcast='unsigned')
transactions_categories['subcategory_min_units_block'] = pd.to_numeric(transactions_categories_blocks.groupby(['subcategory'])['subcategory_block_units'].transform(np.min), downcast='unsigned')
transactions_categories['subcategory_max_units_day'] = pd.to_numeric(transactions_categories.groupby(['subcategory'])['subcategory_day_units'].transform(np.max), downcast='unsigned')
transactions_categories['subcategory_min_units_day'] = pd.to_numeric(transactions_categories.groupby(['subcategory'])['subcategory_day_units'].transform(np.min), downcast='unsigned')

In [37]:
transactions_categories['subcategory_turnover'] = pd.to_numeric(transactions_categories.groupby(['subcategory'])['turnover'].transform(np.sum), downcast='unsigned')
transactions_categories['subcategory_mean_turnover_block'] = pd.to_numeric(transactions_categories_blocks.groupby(['subcategory'])['subcategory_block_turnover'].transform(np.mean), downcast='float') 
transactions_categories['subcategory_day_turnover'] = pd.to_numeric(transactions_categories.groupby(['subcategory','date'])['turnover'].transform(np.sum), downcast='unsigned')
transactions_categories['subcategory_mean_turnover_day'] = pd.to_numeric(transactions_categories.groupby(['subcategory'])['subcategory_day_turnover'].transform(np.mean), downcast='float') 
transactions_categories['subcategory_max_turnover_block'] = pd.to_numeric(transactions_categories_blocks.groupby(['subcategory'])['subcategory_block_turnover'].transform(np.max), downcast='unsigned')
transactions_categories['subcategory_min_turnover_block'] = pd.to_numeric(transactions_categories_blocks.groupby(['subcategory'])['subcategory_block_turnover'].transform(np.min), downcast='unsigned')
transactions_categories['subcategory_max_turnover_day'] = pd.to_numeric(transactions_categories.groupby(['subcategory'])['subcategory_day_turnover'].transform(np.max), downcast='unsigned')
transactions_categories['subcategory_min_turnover_day'] = pd.to_numeric(transactions_categories.groupby(['subcategory'])['subcategory_day_turnover'].transform(np.min), downcast='unsigned')

In [38]:
transactions_categories['category_share_of_total_units'] = pd.to_numeric(transactions_categories['category_units'] * 100 / total_sales , downcast='float') 
transactions_categories['category_share_of_total_turnover'] = pd.to_numeric(transactions_categories['category_turnover']* 100 / total_turnover, downcast='float') 

transactions_categories['subcategory_units'] = pd.to_numeric(transactions_categories.groupby("subcategory")['item_cnt_day'].transform(np.sum), downcast='unsigned')
transactions_categories['subcategory_share_of_total_units'] = pd.to_numeric(transactions_categories['subcategory_units'] * 100 / total_sales, downcast='float') 
transactions_categories['subcategory_turnover'] = pd.to_numeric(transactions_categories.groupby("subcategory")['turnover'].transform(np.sum), downcast='unsigned')
transactions_categories['subcategory_share_of_total_turnover'] = pd.to_numeric(transactions_categories['subcategory_turnover']* 100 / total_turnover, downcast='float') 

In [39]:
subcategory_first_two_blocks_units = transactions_categories.groupby(['subcategory','is_first_two_blocks'], as_index=False)['item_cnt_day'].sum()
subcategory_first_two_blocks_units = subcategory_first_two_blocks_units[subcategory_first_two_blocks_units['is_first_two_blocks'] == True].set_index('subcategory').iloc[:,1]
transactions_categories['subcategory_first_two_blocks_units'] = pd.to_numeric(transactions_categories['subcategory'].map(subcategory_first_two_blocks_units), downcast='unsigned')

subcategory_last_two_blocks_units = transactions_categories.groupby(['subcategory','is_last_two_blocks'], as_index=False)['item_cnt_day'].sum()
subcategory_last_two_blocks_units = subcategory_last_two_blocks_units[subcategory_last_two_blocks_units['is_last_two_blocks'] == True].set_index('subcategory').iloc[:,1]
transactions_categories['subcategory_last_two_blocks_units'] = pd.to_numeric(transactions_categories['subcategory'].map(subcategory_last_two_blocks_units), downcast='unsigned')

transactions_categories['subcategory_fluctuation_units_first_last_blocks'] =  pd.to_numeric(((transactions_categories['subcategory_first_two_blocks_units'] - transactions_categories['subcategory_last_two_blocks_units'] ) / \
                                                             transactions_categories['subcategory_first_two_blocks_units']) * 100 * -1, downcast='float') 


subcategory_first_two_blocks_mean_price = transactions_categories.groupby(['subcategory','is_first_two_blocks'], as_index=False)['item_price'].mean()
subcategory_first_two_blocks_mean_price = subcategory_first_two_blocks_mean_price[subcategory_first_two_blocks_mean_price['is_first_two_blocks'] == True].set_index('subcategory').iloc[:,1]
transactions_categories['subcategory_first_two_blocks_mean_price'] = pd.to_numeric(transactions_categories['subcategory'].map(subcategory_first_two_blocks_mean_price), downcast='unsigned')

subcategory_last_two_blocks_mean_price = transactions_categories.groupby(['subcategory','is_last_two_blocks'], as_index=False)['item_price'].mean()
subcategory_last_two_blocks_mean_price = subcategory_last_two_blocks_mean_price[subcategory_last_two_blocks_mean_price['is_last_two_blocks'] == True].set_index('subcategory').iloc[:,1]
transactions_categories['subcategory_last_two_blocks_mean_price'] = pd.to_numeric(transactions_categories['subcategory'].map(subcategory_last_two_blocks_mean_price), downcast='unsigned')

transactions_categories['subcategory_fluctuation_price_first_last_blocks'] =  pd.to_numeric(((transactions_categories['subcategory_first_two_blocks_mean_price'] - transactions_categories['subcategory_last_two_blocks_mean_price'] ) / \
                                                             transactions_categories['subcategory_first_two_blocks_mean_price'])  * 100 * -1, downcast='float') 

In [40]:

transactions_categories.to_pickle("pickled/transactions_categories")
transactions_categories_blocks.to_pickle("pickled/transactions_categories_blocks")

del transactions_categories
del transactions_categories_blocks
gc.collect()

221

#SHOP

-UNITS
shop_units
shop_block_units
shop_mean_units_block
shop_day_units
shop_mean_units_day
shop_max_units_block
shop_min_units_block
shop_max_units_day
shop_min_units_day

-TURNOVER
shop_turnover
shop_block_turnover
shop_mean_turnover_block
shop_day_turnover
shop_mean_turnover_day
shop_max_turnover_block
shop_min_turnover_block
shop_max_turnover_day
shop_min_turnover_day

-PRICE
shop_mean_price
shop_mean_price_block


-TREND
shop_first_two_blocks_units
shop_last_two_blocks_units
shop_fluctuation_units_first_last_blocks
shop_first_two_blocks_mean_price
shop_last_two_blocks_mean_price
shop_fluctuation_price_first_last_blocks

-ENCODINGS
shop_share_of_total_units
shop_share_of_total_gross

-MISC
shop_ids_TC
shop_ids_TRK
shop_ids_SEC
shop_ids_shopping_center
shop_ids_moscow

-CATEGORY
shop_top_category_units
shop_top_category_turnover
shop_top_subcategory_units
shop_top_subcategory_turnover

In [41]:
gc.collect()
transactions_shops = transactions.copy()
transactions_shops_blocks = transactions.copy()

In [42]:
shop_areas = {}
for i in range(0,2):
    shop_areas[i] = "Yakutsk"
shop_areas[2] = "Adygea"
shop_areas[3] = "Balashikha"
shop_areas[4] = "Volga"
shop_areas[5] = "Vologda"
for i in range(6,9):
    shop_areas[i] = "Voronezh"
shop_areas[9] = "Outbound Trading"
for i in range(10,12):
    shop_areas[i] = "Zhukovsky"
shop_areas[12] = "Online store emergency"
for i in range(13,15):
    shop_areas[i] = "Kazan"
shop_areas[15] = "Kaluga"
shop_areas[16] = "Kolomna"
for i in range(17,19):
    shop_areas[i] = "Krasnoyarsk"
shop_areas[19] = "Kursk"
for i in range(20,33):
    shop_areas[i] = "Moscow"
shop_areas[33] = "Mytishchi"
for i in range(34,36):
    shop_areas[i] = "N.Novgorod"
for i in range(36,38):
    shop_areas[i] = "Novosibirsk"
shop_areas[38] = "Omsk"
for i in range(39,42):
    shop_areas[i] = "RostovNaDonu"
for i in range(42,44):
    shop_areas[i] = "St. Petersburg"
for i in range(44,46):
    shop_areas[i] = "Samara"
shop_areas[46] = "Sergiev Posad"
shop_areas[47] = "Surgut"
shop_areas[48] = "Tomsk"
for i in range(49,52):
    shop_areas[i] = "Tyumen TC"
for i in range(52,54):
    shop_areas[i] = "Ufa"
shop_areas[54] = "Khimki"
shop_areas[55] = "Digital warehouse"
shop_areas[56] = "Chekhov"
for i in range(57,59):
    shop_areas[i] = "Yakutsk"
shop_areas[59] = "Yaroslavl"

transactions_shops['area'] = transactions_shops['shop_id'].apply(lambda x: shop_areas[x]).astype('category')

transactions_shops_blocks['area'] = transactions_shops['shop_id'].apply(lambda x: shop_areas[x]).astype('category')



In [43]:
transactions_shops_blocks['shop_block_units'] = pd.to_numeric(transactions_shops_blocks.groupby(['shop_id','date_block_num'])['item_cnt_day'].transform(np.sum), downcast='unsigned')
transactions_shops_blocks['shop_block_turnover'] = pd.to_numeric(transactions_shops_blocks.groupby(['shop_id','date_block_num'])['turnover'].transform(np.sum), downcast='unsigned')
transactions_shops_blocks['shop_mean_price_block'] = pd.to_numeric(transactions_shops_blocks.groupby(['shop_id', 'date_block_num'])['item_price'].transform(np.mean), downcast='float') 
transactions_shops_blocks['area_block_units'] = pd.to_numeric(transactions_shops_blocks.groupby(['area','date_block_num'])['item_cnt_day'].transform(np.sum), downcast='unsigned')
transactions_shops_blocks['area_block_turnover'] = pd.to_numeric(transactions_shops_blocks.groupby(['area','date_block_num'])['turnover'].transform(np.sum), downcast='unsigned')
transactions_shops_blocks['area_mean_price_block'] = pd.to_numeric(transactions_shops_blocks.groupby(['area', 'date_block_num'])['item_price'].transform(np.mean), downcast='float') 


In [44]:
transactions_shops['shop_units'] = pd.to_numeric(transactions_shops.groupby(['shop_id'])['item_cnt_day'].transform(np.sum), downcast='unsigned')
transactions_shops['shop_mean_units_block'] = pd.to_numeric(transactions_shops_blocks.groupby(['shop_id'])['shop_block_units'].transform(np.mean), downcast='float') 
transactions_shops['shop_day_units'] = pd.to_numeric(transactions_shops.groupby(['shop_id','date'])['item_cnt_day'].transform(np.sum), downcast='unsigned')
transactions_shops['shop_mean_units_day'] = pd.to_numeric(transactions_shops.groupby(['shop_id'])['shop_day_units'].transform(np.mean), downcast='float') 
transactions_shops['shop_max_units_block'] = pd.to_numeric(transactions_shops_blocks.groupby(['shop_id'])['shop_block_units'].transform(np.max), downcast='unsigned')
transactions_shops['shop_min_units_block'] = pd.to_numeric(transactions_shops_blocks.groupby(['shop_id'])['shop_block_units'].transform(np.min), downcast='unsigned')
transactions_shops['shop_max_units_day'] = pd.to_numeric(transactions_shops.groupby(['shop_id'])['shop_day_units'].transform(np.max), downcast='unsigned')
transactions_shops['shop_min_units_day'] = pd.to_numeric(transactions_shops.groupby(['shop_id'])['shop_day_units'].transform(np.min), downcast='unsigned')

In [45]:
transactions_shops['shop_turnover'] = pd.to_numeric(transactions_shops.groupby(['shop_id'])['turnover'].transform(np.sum), downcast='unsigned')
transactions_shops['shop_mean_turnover_block'] = pd.to_numeric(transactions_shops_blocks.groupby(['shop_id'])['shop_block_turnover'].transform(np.mean), downcast='float') 
transactions_shops['shop_day_turnover'] = pd.to_numeric(transactions_shops.groupby(['shop_id','date'])['turnover'].transform(np.sum), downcast='unsigned')
transactions_shops['shop_mean_turnover_day'] = pd.to_numeric(transactions_shops.groupby(['shop_id'])['shop_day_turnover'].transform(np.mean), downcast='float') 
transactions_shops['shop_max_turnover_block'] = pd.to_numeric(transactions_shops_blocks.groupby(['shop_id'])['shop_block_turnover'].transform(np.max), downcast='unsigned')
transactions_shops['shop_min_turnover_block'] = pd.to_numeric(transactions_shops_blocks.groupby(['shop_id'])['shop_block_turnover'].transform(np.min), downcast='unsigned')
transactions_shops['shop_max_turnover_day'] = pd.to_numeric(transactions_shops.groupby(['shop_id'])['shop_day_turnover'].transform(np.max), downcast='unsigned')
transactions_shops['shop_min_turnover_day'] = pd.to_numeric(transactions_shops.groupby(['shop_id'])['shop_day_turnover'].transform(np.min), downcast='unsigned')

In [46]:
transactions_shops['shop_mean_price'] = pd.to_numeric(transactions_shops.groupby('shop_id')['item_price'].transform(np.mean), downcast='float') 


In [47]:
                                                   


shop_first_two_blocks_mean_price = transactions_shops.groupby(['shop_id','is_first_two_blocks'], as_index=False)['item_price'].mean()
shop_first_two_blocks_mean_price = shop_first_two_blocks_mean_price[shop_first_two_blocks_mean_price['is_first_two_blocks'] == True].set_index('shop_id').iloc[:,1]
transactions_shops['shop_first_two_blocks_mean_price'] = pd.to_numeric(transactions_shops['shop_id'].map(shop_first_two_blocks_mean_price), downcast='unsigned')

shop_last_two_blocks_mean_price = transactions_shops.groupby(['shop_id','is_last_two_blocks'], as_index=False)['item_price'].mean()
shop_last_two_blocks_mean_price = shop_last_two_blocks_mean_price[shop_last_two_blocks_mean_price['is_last_two_blocks'] == True].set_index('shop_id').iloc[:,1]
transactions_shops['shop_last_two_blocks_mean_price'] = pd.to_numeric(transactions_shops['shop_id'].map(shop_last_two_blocks_mean_price), downcast='unsigned')

transactions_shops['shop_fluctuation_price_first_last_blocks'] =  pd.to_numeric(((transactions_shops['shop_first_two_blocks_mean_price'] - transactions_shops['shop_last_two_blocks_mean_price'] ) / \
                                                             transactions_shops['shop_first_two_blocks_mean_price'])  * 100 * -1, downcast='float') 

In [48]:
transactions_shops['shop_share_of_units'] = pd.to_numeric(transactions_shops['shop_units'] * 100 / total_sales, downcast='float') 
transactions_shops['shop_share_of_turnover'] = pd.to_numeric(transactions_shops['shop_turnover'] * 100 / total_turnover, downcast='float') 

In [49]:
shop_ids_TC = [1,2,13,14,16,23,24,26,28,31,37,38,42,43,44,46,50,54,58]
shop_ids_TRK = [3,33,39,40]
shop_ids_SEC = [7,34,36,47,48,49,56]
shop_ids_shopping_center = [4,5,8,15,17,18,19,27,29,30,32,41,45,51,53,59]
shop_ids_moscow = list(range(20,33))


transactions_shops['shop_TC'] = transactions_shops['shop_id'].isin(shop_ids_TC)
transactions_shops['shop_TRK'] = transactions_shops['shop_id'].isin(shop_ids_TRK)
transactions_shops['shop_SEC'] = transactions_shops['shop_id'].isin(shop_ids_SEC)
transactions_shops['shop_shopping_center'] = transactions_shops['shop_id'].isin(shop_ids_shopping_center)
transactions_shops['shop_moscow'] = transactions_shops['shop_id'].isin(shop_ids_moscow)

In [50]:
a = transactions_shops.groupby(['shop_id', 'item_category_id'],as_index=False)['item_cnt_day'].sum()

b = pd.DataFrame(transactions_shops.groupby(['shop_id', 'item_category_id'],as_index=False)['item_cnt_day'].sum()\
                  .groupby(['shop_id'])['item_cnt_day'].max()).reset_index()

c = b.merge(a, on=['shop_id', 'item_cnt_day'],how='left')[['shop_id', 'item_category_id']].rename(columns={'item_category_id':'max_category_units'})

transactions_shops = transactions_shops.merge(c, on=['shop_id'], how='left')

transactions_shops['max_category_units'] = pd.to_numeric(transactions_shops['max_category_units'], downcast='unsigned')

In [51]:
a = transactions_shops.groupby(['shop_id', 'item_category_id'],as_index=False)['turnover'].sum()

b = pd.DataFrame(transactions_shops.groupby(['shop_id', 'item_category_id'],as_index=False)['turnover'].sum()\
                  .groupby(['shop_id'])['turnover'].max()).reset_index()

c = b.merge(a, on=['shop_id', 'turnover'],how='left')[['shop_id', 'item_category_id']].rename(columns={'item_category_id':'max_category_turnover'})

transactions_shops = transactions_shops.merge(c, on=['shop_id'], how='left')
transactions_shops['max_category_turnover'] = pd.to_numeric(transactions_shops['max_category_turnover'], downcast='unsigned')

-AREA
area



-UNITS
area_units
area_block_units
area_mean_units_block
area_day_units
area_mean_units_day
area_max_units_block
area_min_units_block
area_max_units_day
area_min_units_day

-TURNOVER
area_turnover
area_block_turnover
area_mean_turnover_block
area_day_turnover
area_mean_turnover_day
area_max_turnover_block
area_min_turnover_block
area_max_turnover_day
area_min_turnover_day

-PRICE
area_mean_price
area_mean_price_block


-TREND
area_first_two_blocks_units
area_last_two_blocks_units
area_fluctuation_units_first_last_blocks
area_first_two_blocks_mean_price
area_last_two_blocks_mean_price
area_fluctuation_price_first_last_blocks

-ENCODINGS
area_share_of_total_units
area_share_of_total_gross

In [52]:
transactions_shops['area_units'] = pd.to_numeric(transactions_shops.groupby(['area'])['item_cnt_day'].transform(np.sum), downcast='unsigned')
transactions_shops['area_mean_units_block'] = pd.to_numeric(transactions_shops_blocks.groupby(['area'])['area_block_units'].transform(np.mean), downcast='float') 
transactions_shops['area_day_units'] = pd.to_numeric(transactions_shops.groupby(['area','date'])['item_cnt_day'].transform(np.sum), downcast='unsigned')
transactions_shops['area_mean_units_day'] = pd.to_numeric(transactions_shops.groupby(['area'])['area_day_units'].transform(np.mean), downcast='float') 
transactions_shops['area_max_units_block'] = pd.to_numeric(transactions_shops_blocks.groupby(['area'])['area_block_units'].transform(np.max), downcast='unsigned')
transactions_shops['area_min_units_block'] = pd.to_numeric(transactions_shops_blocks.groupby(['area'])['area_block_units'].transform(np.min), downcast='unsigned')
transactions_shops['area_max_units_day'] = pd.to_numeric(transactions_shops.groupby(['area'])['area_day_units'].transform(np.max), downcast='unsigned')
transactions_shops['area_min_units_day'] = pd.to_numeric(transactions_shops.groupby(['area'])['area_day_units'].transform(np.min), downcast='unsigned')

In [53]:
transactions_shops['area_turnover'] = pd.to_numeric(transactions_shops.groupby(['area'])['turnover'].transform(np.sum), downcast='unsigned')
transactions_shops['area_mean_turnover_block'] = pd.to_numeric(transactions_shops_blocks.groupby(['area'])['area_block_turnover'].transform(np.mean), downcast='float') 
transactions_shops['area_day_turnover'] = pd.to_numeric(transactions_shops.groupby(['area','date'])['turnover'].transform(np.sum), downcast='unsigned')
transactions_shops['area_mean_turnover_day'] = pd.to_numeric(transactions_shops.groupby(['area'])['area_day_turnover'].transform(np.mean), downcast='float') 
transactions_shops['area_max_turnover_block'] = pd.to_numeric(transactions_shops_blocks.groupby(['area'])['area_block_turnover'].transform(np.max), downcast='unsigned')
transactions_shops['area_min_turnover_block'] = pd.to_numeric(transactions_shops_blocks.groupby(['area'])['area_block_turnover'].transform(np.min), downcast='unsigned')
transactions_shops['area_max_turnover_day'] = pd.to_numeric(transactions_shops.groupby(['area'])['area_day_turnover'].transform(np.max), downcast='unsigned')
transactions_shops['area_min_turnover_day'] = pd.to_numeric(transactions_shops.groupby(['area'])['area_day_turnover'].transform(np.min), downcast='unsigned')

In [54]:
transactions_shops['area_mean_price'] = pd.to_numeric(transactions_shops.groupby('area')['item_price'].transform(np.mean), downcast='float') 


In [55]:
area_first_two_blocks_units = transactions_shops.groupby(['area','is_first_two_blocks'], as_index=False)['item_cnt_day'].sum()
area_first_two_blocks_units = area_first_two_blocks_units[area_first_two_blocks_units['is_first_two_blocks'] == True].set_index('area').iloc[:,1]
transactions_shops['area_first_two_blocks_units'] = pd.to_numeric(transactions_shops['area'].map(area_first_two_blocks_units), downcast='unsigned')

area_last_two_blocks_units = transactions_shops.groupby(['area','is_last_two_blocks'], as_index=False)['item_cnt_day'].sum()
area_last_two_blocks_units = area_last_two_blocks_units[area_last_two_blocks_units['is_last_two_blocks'] == True].set_index('area').iloc[:,1]
transactions_shops['area_last_two_blocks_units'] = pd.to_numeric(transactions_shops['area'].map(area_last_two_blocks_units), downcast='unsigned')

transactions_shops['area_fluctuation_units_first_last_blocks'] =  pd.to_numeric(((transactions_shops['area_first_two_blocks_units'] - transactions_shops['area_last_two_blocks_units'] ) / \
                                                             transactions_shops['area_first_two_blocks_units']) * 100 * -1, downcast='float') 


area_first_two_blocks_mean_price = transactions_shops.groupby(['area','is_first_two_blocks'], as_index=False)['item_price'].mean()
area_first_two_blocks_mean_price = area_first_two_blocks_mean_price[area_first_two_blocks_mean_price['is_first_two_blocks'] == True].set_index('area').iloc[:,1]
transactions_shops['area_first_two_blocks_mean_price'] = pd.to_numeric(transactions_shops['area'].map(area_first_two_blocks_mean_price), downcast='unsigned')

area_last_two_blocks_mean_price = transactions_shops.groupby(['area','is_last_two_blocks'], as_index=False)['item_price'].mean()
area_last_two_blocks_mean_price = area_last_two_blocks_mean_price[area_last_two_blocks_mean_price['is_last_two_blocks'] == True].set_index('area').iloc[:,1]
transactions_shops['area_last_two_blocks_mean_price'] = pd.to_numeric(transactions_shops['area'].map(area_last_two_blocks_mean_price), downcast='unsigned')

transactions_shops['area_fluctuation_price_first_last_blocks'] =  pd.to_numeric(((transactions_shops['area_first_two_blocks_mean_price'] - transactions_shops['area_last_two_blocks_mean_price'] ) / \
                                                             transactions_shops['area_first_two_blocks_mean_price'])  * 100 * -1, downcast='float') 

In [56]:

transactions_shops.to_pickle("pickled/transactions_shops")
transactions_shops_blocks.to_pickle("pickled/transactions_shops_blocks")


del transactions_shops
del transactions_shops_blocks
gc.collect()

182

shop_category


-UNITS
shop_category_units
shop_category_block_units
shop_category_mean_units_block
shop_category_day_units
shop_category_mean_units_day
shop_category_max_units_block
shop_category_min_units_block
shop_category_max_units_day
shop_category_min_units_day

-TURNOVER
shop_category_turnover
shop_category_block_turnover
shop_category_mean_turnover_block
shop_category_day_turnover
shop_category_mean_turnover_day
shop_category_max_turnover_block
shop_category_min_turnover_block
shop_category_max_turnover_day
shop_category_min_turnover_day

-PRICE
shop_category_mean_price
shop_category_mean_price_block


-TREND
shop_category_first_two_blocks_units
shop_category_last_two_blocks_units
shop_category_fluctuation_units_first_last_blocks
shop_category_first_two_blocks_mean_price
shop_category_last_two_blocks_mean_price
shop_category_fluctuation_price_first_last_blocks

-ENCODINGS
shop_category_share_of_total_units
shop_category_share_of_total_gross

In [57]:
gc.collect()
transactions_shops_categories = transactions.copy()
transactions_shops_categories_blocks = transactions.copy()

In [58]:
transactions_shops_categories_blocks['shop_category_block_units'] = pd.to_numeric(transactions_shops_categories_blocks.groupby(['shop_id','item_category_id','date_block_num'])['item_cnt_day'].transform(np.sum), downcast='unsigned')
transactions_shops_categories_blocks['shop_category_block_turnover'] = pd.to_numeric(transactions_shops_categories_blocks.groupby(['shop_id','item_category_id','date_block_num'])['turnover'].transform(np.sum), downcast='unsigned')
transactions_shops_categories_blocks['shop_category_mean_price_block'] = pd.to_numeric(transactions_shops_categories_blocks.groupby(['shop_id','item_category_id', 'date_block_num'])['item_price'].transform(np.mean), downcast='float') 

In [59]:
transactions_shops_categories['shop_category_units'] = pd.to_numeric(transactions_shops_categories.groupby(['shop_id','item_category_id'])['item_cnt_day'].transform(np.sum), downcast='unsigned')
transactions_shops_categories['shop_category_mean_units_block'] = pd.to_numeric(transactions_shops_categories_blocks.groupby(['shop_id','item_category_id'])['shop_category_block_units'].transform(np.mean), downcast='float') 
transactions_shops_categories['shop_category_day_units'] = pd.to_numeric(transactions_shops_categories.groupby(['shop_id','item_category_id','date'])['item_cnt_day'].transform(np.sum), downcast='unsigned')
transactions_shops_categories['shop_category_mean_units_day'] = pd.to_numeric(transactions_shops_categories.groupby(['shop_id','item_category_id'])['shop_category_day_units'].transform(np.mean), downcast='float') 
transactions_shops_categories['shop_category_max_units_block'] = pd.to_numeric(transactions_shops_categories_blocks.groupby(['shop_id','item_category_id'])['shop_category_block_units'].transform(np.max), downcast='unsigned')
transactions_shops_categories['shop_category_min_units_block'] = pd.to_numeric(transactions_shops_categories_blocks.groupby(['shop_id','item_category_id'])['shop_category_block_units'].transform(np.min), downcast='unsigned')
transactions_shops_categories['shop_category_max_units_day'] = pd.to_numeric(transactions_shops_categories.groupby(['shop_id','item_category_id'])['shop_category_day_units'].transform(np.max), downcast='unsigned')
transactions_shops_categories['shop_category_min_units_day'] = pd.to_numeric(transactions_shops_categories.groupby(['shop_id','item_category_id'])['shop_category_day_units'].transform(np.min), downcast='unsigned')


In [60]:
transactions_shops_categories['shop_category_turnover'] = pd.to_numeric(transactions_shops_categories.groupby(['shop_id','item_category_id'])['turnover'].transform(np.sum), downcast='unsigned')
transactions_shops_categories['shop_category_mean_turnover_block'] = pd.to_numeric(transactions_shops_categories_blocks.groupby(['shop_id','item_category_id'])['shop_category_block_turnover'].transform(np.mean), downcast='float') 
transactions_shops_categories['shop_category_day_turnover'] = pd.to_numeric(transactions_shops_categories.groupby(['shop_id','item_category_id','date'])['turnover'].transform(np.sum), downcast='unsigned')
transactions_shops_categories['shop_category_mean_turnover_day'] = pd.to_numeric(transactions_shops_categories.groupby(['shop_id','item_category_id'])['shop_category_day_turnover'].transform(np.mean), downcast='float') 
transactions_shops_categories['shop_category_max_turnover_block'] = pd.to_numeric(transactions_shops_categories_blocks.groupby(['shop_id','item_category_id'])['shop_category_block_turnover'].transform(np.max), downcast='unsigned')
transactions_shops_categories['shop_category_min_turnover_block'] = pd.to_numeric(transactions_shops_categories_blocks.groupby(['shop_id','item_category_id'])['shop_category_block_turnover'].transform(np.min), downcast='unsigned')
transactions_shops_categories['shop_category_max_turnover_day'] = pd.to_numeric(transactions_shops_categories.groupby(['shop_id','item_category_id'])['shop_category_day_turnover'].transform(np.max), downcast='unsigned')
transactions_shops_categories['shop_category_min_turnover_day'] = pd.to_numeric(transactions_shops_categories.groupby(['shop_id','item_category_id'])['shop_category_day_turnover'].transform(np.min), downcast='unsigned')

In [61]:
transactions_shops_categories['shop_category_mean_price'] = pd.to_numeric(transactions_shops_categories.groupby(['shop_id','item_category_id'])['item_price'].transform(np.mean), downcast='float') 


In [62]:

transactions_shops_categories.to_pickle("pickled/transactions_shops_categories")
transactions_shops_categories_blocks.to_pickle("pickled/transactions_shops_categories_blocks")


del transactions_shops_categories
del transactions_shops_categories_blocks
gc.collect()

84

In [63]:
###
#DEBUG
###


pd.set_option('display.max_columns', None)  
pd.set_option('display.expand_frame_repr', False)
pd.set_option('max_colwidth', -1)
#transactions.sample(10).sort_values(by=['item_units'], ascending=False)
#transactions[transactions['item_category_id'] == 58].sample(10).sort_values(by=['total_sales_units'], ascending=False)

In [12]:
train_item_ids = transactions['item_id'].unique()
train_shop_ids = transactions['shop_id'].unique()
test_item_ids = test['item_id'].unique()
test_shop_ids = test['shop_id'].unique()
train_blocks = transactions['date_block_num'].unique()

all_item_ids = np.unique(np.append(test_item_ids,train_item_ids))
all_shop_ids = np.unique(np.append(train_shop_ids,test_shop_ids))

In [13]:
len(test_item_ids)

5100

In [14]:
combinations = []
for shop in all_shop_ids:
    #get all article ids ever associated to this shop
    train_ids = transactions[transactions['shop_id'] == shop]['item_id'].unique()
    test_ids = test[test['shop_id'] == shop]['item_id'].unique()
    all_shop = np.unique(np.append(train_ids, test_ids))
    all_shop_combo = [[item, shop, block] for item in all_shop for block in train_blocks]
    for combo in all_shop_combo:
        combinations.append(combo)

In [15]:
len(combinations)

8333930

In [16]:
all_combos = pd.DataFrame(np.unique(np.vstack([combinations]), axis=0), columns=['item_id', 'shop_id', 'date_block_num'])

In [44]:
#all_combos = all_combos.sample(500000)

In [22]:
all_combos.head()

,item_id,shop_id,date_block_num
0,0.0,54.0,12.0
1,0.0,54.0,13.0
2,0.0,54.0,14.0
3,0.0,54.0,15.0
4,0.0,54.0,16.0


In [17]:
all_combos['item_id'] = pd.to_numeric(all_combos['item_id'], downcast='unsigned')
all_combos['shop_id'] = pd.to_numeric(all_combos['shop_id'], downcast='unsigned')
all_combos['date_block_num'] = pd.to_numeric(all_combos['date_block_num'], downcast='unsigned')

In [18]:
len(all_combos)

8333930

In [19]:
all_combos = pd.merge(all_combos, items[['item_id', 'item_category_id']], on='item_id', how='left')

In [20]:
dates = transactions[['date_block_num', 'month', 'year']].drop_duplicates(['date_block_num', 'month', 'year'])

dates_dict = {}

for index,row in dates.iterrows():
    dates_dict[row['date_block_num']] = {"month": row['month'], "year": row['year']}
    
dates_dict

{20: {'month': 9, 'year': 2014},
 15: {'month': 4, 'year': 2014},
 18: {'month': 7, 'year': 2014},
 19: {'month': 8, 'year': 2014},
 21: {'month': 10, 'year': 2014},
 22: {'month': 11, 'year': 2014},
 23: {'month': 12, 'year': 2014},
 24: {'month': 1, 'year': 2015},
 27: {'month': 4, 'year': 2015},
 25: {'month': 2, 'year': 2015},
 12: {'month': 1, 'year': 2014},
 14: {'month': 3, 'year': 2014},
 16: {'month': 5, 'year': 2014},
 17: {'month': 6, 'year': 2014},
 13: {'month': 2, 'year': 2014},
 26: {'month': 3, 'year': 2015},
 28: {'month': 5, 'year': 2015},
 29: {'month': 6, 'year': 2015},
 30: {'month': 7, 'year': 2015},
 31: {'month': 8, 'year': 2015},
 32: {'month': 9, 'year': 2015},
 33: {'month': 10, 'year': 2015}}

In [21]:
all_combos['month'] = pd.to_numeric(all_combos['date_block_num'].apply(lambda block: dates_dict[block]['month']), downcast='unsigned')
all_combos['year'] = pd.to_numeric(all_combos['date_block_num'].apply(lambda block: dates_dict[block]['year']), downcast='unsigned')

In [33]:
def downcast(df, columns, dtypes):
    for column in columns:
        dtype = dtypes[column].kind
        if dtype in ['O', 'b']:
            continue  
        if dtype == 'u':
            df[column] = pd.to_numeric(df[column].astype(int), downcast='unsigned')
        elif dtype == 'i':
            df[column] = pd.to_numeric(df[column].astype(int), downcast='signed')
        else:
            df[column] = pd.to_numeric(df[column], downcast='float')

In [34]:
def fillnas(df, columns, dtypes):
    for column in columns:
        dtype = dtypes[column].kind
        if dtype in ['O', 'b']:
            continue  
        df[column].fillna(0, inplace=True)

In [24]:
#del training

In [25]:
transactions_items_columns = ['item_id', 'item_first_block',
       'item_last_block', 'is_first_two_blocks', 'is_last_two_blocks',
       'item_units', 'item_mean_units_block', 'item_day_units',
       'item_mean_units_day', 'item_max_units_block',
       'item_min_units_block', 'item_max_units_day', 'item_min_units_day',
       'item_turnover', 'item_mean_turnover_block', 'item_day_turnover',
       'item_mean_turnover_day', 'item_max_turnover_block',
       'item_min_turnover_block', 'item_max_turnover_day',
       'item_min_turnover_day', 'item_days_of_activity',
       'item_blocks_of_activity', 'item_days_since_start',
       'item_mean_day_between_activity',
       'item_longest_stretch_days_without_activity',
       'item_longest_stretch_blocks_without_activity',
       'item_longest_stretch_block_with_activity',
       'item_number_of_consecutive_days_with_activity',
       'item_days_between_start_and_first_activity',
       'item_blocks_between_start_and_first_activity', 'item_first_day',
       'item_last_day', 'item_activity_on_all_blocks', 'item_mean_price',
       'item_min_price', 'item_max_price', 'item_number_different_prices',
       'item_price_amplitude', 
       'item_deviation_mean_category_price',
       'item_first_two_blocks_units', 'item_last_two_blocks_units',
       'item_fluctuation_units_first_last_blocks',
       'item_first_two_blocks_mean_price',
       'item_last_two_blocks_mean_price',
       'item_fluctuation_price_first_last_blocks',
       'item_share_of_total_units', 'item_share_of_total_turnover']

In [26]:
transactions_items = pd.read_pickle("pickled/transactions_items")
transactions_items_dtypes = transactions_items.dtypes
training = pd.merge(all_combos, transactions_items[transactions_items_columns].drop_duplicates('item_id'), on=['item_id'], how='left', copy=False)

del transactions_items
fillnas(training, transactions_items_columns, transactions_items_dtypes)
downcast(training, transactions_items_columns, transactions_items_dtypes)
gc.collect()

21

In [26]:
transactions_items_blocks_columns =  [ 'date_block_num', 'item_id', 'item_block_units',
 'item_block_turnover',
 'item_mean_price_block']

In [28]:
transactions_items_blocks = pd.read_pickle("pickled/transactions_items_blocks")
transactions_items_blocks_dtypes = transactions_items_blocks.dtypes

training = pd.merge(training, transactions_items_blocks[transactions_items_blocks_columns]\
                    .drop_duplicates(['item_id', 'date_block_num']), on=['item_id','date_block_num'], how='left', copy=False)

del transactions_items_blocks
training.fillna(0, inplace=True)
downcast(training, transactions_items_blocks_columns, transactions_items_blocks_dtypes)
gc.collect()

14

In [27]:
transactions_categories_columns = [
       'item_category_id',
       'category_units', 'category_mean_units_block',
       'category_day_units', 'category_mean_units_day',
       'category_max_units_block', 'category_min_units_block',
       'category_max_units_day', 'category_min_units_day',
       'category_turnover', 'category_mean_turnover_block',
       'category_day_turnover', 'category_mean_turnover_day',
       'category_max_turnover_block', 'category_min_turnover_block',
       'category_max_turnover_day', 'category_min_turnover_day',
       'category_mean_price', 'category_min_price', 'category_max_price',
       'category_first_two_blocks_units',
       'category_last_two_blocks_units',
       'category_fluctuation_units_first_last_blocks',
       'category_first_two_blocks_mean_price',
       'category_last_two_blocks_mean_price',
       'category_fluctuation_price_first_last_blocks', 'subcategory',
       'video_game', 'gaming_old_gen', 'gaming_new_gen', 'pc_games',
       'payment_cards', 'movies', 'movies_niche', 'books', 'music',
       'music_CD', 'music_vinyl', 'gifts', 'software',
       'subcategory_units',
       'subcategory_mean_units_block', 'subcategory_day_units',
       'subcategory_mean_units_day', 'subcategory_max_units_block',
       'subcategory_min_units_block', 'subcategory_max_units_day',
       'subcategory_min_units_day', 'subcategory_turnover', 'subcategory_mean_turnover_block',
       'subcategory_day_turnover', 'subcategory_mean_turnover_day',
       'subcategory_max_turnover_block', 'subcategory_min_turnover_block',
       'subcategory_max_turnover_day', 'subcategory_min_turnover_day',
       'category_share_of_total_units',
       'category_share_of_total_turnover',
       'subcategory_share_of_total_units',
       'subcategory_share_of_total_turnover',
       'subcategory_first_two_blocks_units',
       'subcategory_last_two_blocks_units',
       'subcategory_fluctuation_units_first_last_blocks',
       'subcategory_first_two_blocks_mean_price',
       'subcategory_last_two_blocks_mean_price',
       'subcategory_fluctuation_price_first_last_blocks']


In [31]:
transactions_categories = pd.read_pickle("pickled/transactions_categories")
transactions_categories_dtypes = transactions_categories.dtypes
training = pd.merge(training, transactions_categories[transactions_categories_columns]\
                    .drop_duplicates('item_category_id'), on=['item_category_id'], how='left', copy=False)

del transactions_categories
fillnas(training, transactions_categories_columns, transactions_categories_dtypes)
downcast(training, transactions_categories_columns, transactions_categories_dtypes)
gc.collect()

14

In [28]:
transactions_categories_blocks_columns = ['item_category_id', 'date_block_num', 'category_block_units',
 'category_block_turnover',
 'category_mean_price_block', 'subcategory_block_units',
 'subcategory_block_turnover',
 'subcategory_mean_price_block']

In [33]:
transactions_categories_blocks = pd.read_pickle("pickled/transactions_categories_blocks")
transactions_categories_blocks_dtypes = transactions_categories_blocks.dtypes
training = pd.merge(training, transactions_categories_blocks[transactions_categories_blocks_columns]\
                    .drop_duplicates(['item_category_id', 'date_block_num']), on=['item_category_id', 'date_block_num'], how='left', copy=False)

del transactions_categories_blocks
fillnas(training, transactions_categories_blocks_columns, transactions_categories_blocks_dtypes)
downcast(training, transactions_categories_blocks_columns, transactions_categories_blocks_dtypes)
gc.collect()

14

In [29]:
transactions_shops_columns = ['shop_id', 
       'shop_units', 'shop_mean_units_block', 'shop_day_units',
       'shop_mean_units_day', 'shop_max_units_block',
       'shop_min_units_block', 'shop_max_units_day', 'shop_min_units_day',
       'shop_turnover', 'shop_mean_turnover_block', 'shop_day_turnover',
       'shop_mean_turnover_day', 'shop_max_turnover_block',
       'shop_min_turnover_block', 'shop_max_turnover_day',
       'shop_min_turnover_day', 'shop_mean_price',
       'shop_first_two_blocks_units', 'shop_last_two_blocks_units',
       'shop_fluctuation_units_first_last_blocks',
       'shop_first_two_blocks_mean_price',
       'shop_last_two_blocks_mean_price',
       'shop_fluctuation_price_first_last_blocks', 'shop_share_of_units',
       'shop_share_of_turnover', 'shop_TC', 'shop_TRK', 'shop_SEC',
       'shop_shopping_center', 'shop_moscow', 'max_category_units',
       'max_category_turnover', 'area', 'area_units',
       'area_mean_units_block', 'area_day_units', 'area_mean_units_day',
       'area_max_units_block', 'area_min_units_block',
       'area_max_units_day', 'area_min_units_day', 'area_turnover',
     'area_mean_turnover_block',
       'area_day_turnover', 'area_mean_turnover_day',
       'area_max_turnover_block', 'area_min_turnover_block',
       'area_max_turnover_day', 'area_min_turnover_day',
       'area_mean_price',
       'area_first_two_blocks_units', 'area_last_two_blocks_units',
       'area_fluctuation_units_first_last_blocks',
       'area_first_two_blocks_mean_price',
       'area_last_two_blocks_mean_price',
       'area_fluctuation_price_first_last_blocks']

In [35]:
transactions_shops = pd.read_pickle("pickled/transactions_shops")
transactions_shops_dtypes = transactions_shops.dtypes
training = pd.merge(training, transactions_shops[transactions_shops_columns]\
                    .drop_duplicates('shop_id'), on=['shop_id'], how='left', copy=False)

del transactions_shops
fillnas(training, transactions_shops_columns, transactions_shops_dtypes)
downcast(training, transactions_shops_columns, transactions_shops_dtypes)
gc.collect()

14

In [30]:
transactions_shops_blocks_columns = ['shop_id', 'date_block_num',  'shop_block_units',
 'shop_block_turnover',
 'shop_mean_price_block', 'area_block_units',
 'area_block_turnover',
 'area_mean_price_block']

In [37]:
transactions_shops_blocks = pd.read_pickle("pickled/transactions_shops_blocks")
transactions_shops_blocks_dtypes = transactions_shops_blocks.dtypes
training = pd.merge(training, transactions_shops_blocks[transactions_shops_blocks_columns]\
                    .drop_duplicates(['shop_id', 'date_block_num']), on=['shop_id', 'date_block_num'], how='left', copy=False)

del transactions_shops_blocks
fillnas(training, transactions_shops_blocks_columns, transactions_shops_blocks_dtypes)
downcast(training, transactions_shops_blocks_columns, transactions_shops_blocks_dtypes)
gc.collect()

14

In [31]:
transactions_shops_categories_columns = [ 'shop_id', 
       'item_category_id',
       'shop_category_units', 'shop_category_mean_units_block',
       'shop_category_day_units', 'shop_category_mean_units_day',
       'shop_category_max_units_block', 'shop_category_min_units_block',
       'shop_category_max_units_day', 'shop_category_min_units_day',
       'shop_category_turnover', 'shop_category_mean_turnover_block',
       'shop_category_day_turnover', 'shop_category_mean_turnover_day',
       'shop_category_max_turnover_block',
       'shop_category_min_turnover_block',
       'shop_category_max_turnover_day', 'shop_category_min_turnover_day',
       'shop_category_mean_price']

In [39]:
transactions_shops_categories = pd.read_pickle("pickled/transactions_shops_categories")
transactions_shops_categories_dtypes = transactions_shops_categories.dtypes
training = pd.merge(training, transactions_shops_categories[transactions_shops_categories_columns]\
                    .drop_duplicates(['shop_id','item_category_id']), on=['shop_id','item_category_id'], how='left', copy=False)

del transactions_shops_categories
fillnas(training, transactions_shops_categories_columns, transactions_shops_categories_dtypes)
downcast(training, transactions_shops_categories_columns, transactions_shops_categories_dtypes)
gc.collect()

14

In [32]:
transactions_shops_categories_blocks_columns = ['shop_id', 'item_category_id', 'date_block_num',   'shop_category_block_units',
 'shop_category_block_turnover',
 'shop_category_mean_price_block']

In [41]:
transactions_shops_categories_blocks = pd.read_pickle("pickled/transactions_shops_categories_blocks")
transactions_shops_categories_blocks_dtypes = transactions_shops_categories_blocks.dtypes
training = pd.merge(training, transactions_shops_categories_blocks[transactions_shops_categories_blocks_columns]\
                    .drop_duplicates(['shop_id','item_category_id','date_block_num']),\
                    on=['shop_id','item_category_id','date_block_num'], how='left', copy=False)

del transactions_shops_categories_blocks
fillnas(training, transactions_shops_categories_blocks_columns, transactions_shops_categories_blocks_dtypes)
downcast(training, transactions_shops_categories_blocks_columns, transactions_shops_categories_blocks_dtypes)
gc.collect()

14

In [47]:
len(training.drop_duplicates(['shop_id','item_id','date_block_num']))

8333930

In [46]:
len(training)

8333930

In [43]:
training.dtypes

item_id                                       int16
shop_id                                       uint8
date_block_num                                uint8
item_category_id                              uint8
month                                         uint8
year                                         uint16
item_first_block                              uint8
item_last_block                               uint8
is_first_two_blocks                          object
is_last_two_blocks                           object
item_units                                  float32
item_mean_units_block                       float32
item_day_units                                int16
item_mean_units_day                         float32
item_max_units_block                          int16
item_min_units_block                          int16
item_max_units_day                            int16
item_min_units_day                             int8
item_turnover                                 int32
item_mean_tu

In [9]:
#del training
gc.collect()
#training.to_pickle("pickled/training_pre_lags")
#training = pd.read_pickle("pickled/training_pre_lags")

In [4]:
lag_columns = [
 'item_block_units',
 'item_block_turnover',
 'item_mean_price_block',
 'category_block_units',
 'category_block_turnover',
 'category_mean_price_block',
 'subcategory_block_units',
 'subcategory_block_turnover',
 'subcategory_mean_price_block',
 'shop_block_units',
 'shop_block_turnover',
 'shop_mean_price_block',
 'area_block_units',
 'area_block_turnover',
 'area_mean_price_block',
 'shop_category_block_units',
 'shop_category_turnover',
 'shop_category_mean_price_block'
 ]

In [47]:
def downcast_lags(df, lagged_names):
    for lagged_name in lagged_names:
        df[lagged_name].fillna(0,inplace=True)    
    for column in lagged_names:
        if "mean" in column:
            df[column] = pd.to_numeric(df[column], downcast='float')
        else:
            df[column] = pd.to_numeric(df[column].astype(int), downcast='unsigned')
    return df

In [48]:
lags = [1,2,3]
#lags = [1]


def add_lag_features(df, lag_columns, idx_columns):

    gc.collect()
    def lagged_name(lag_column, lag):
        return "%s_lag_%d" % (lag_column, lag)

    merge_columns = ['lagged_block'] + idx_columns

    for lag in lags:
        print(lag)
        lagged = df[['date_block_num'] + idx_columns + lag_columns].copy()
        lagged.rename(columns={'date_block_num':'lagged_block'},inplace=True)
        df['lagged_block'] = df['date_block_num'] - lag
        lagged_names = [lagged_name(c,lag) for c in lag_columns]
        lag_mapping = dict(zip(lag_columns, lagged_names))
        lagged.rename(columns=lag_mapping,inplace=True)
        
        df.set_index(merge_columns, inplace=True)
        #lagged.drop(columns=lag_columns, inplace=True)
        lagged.drop_duplicates(lagged_names+merge_columns, inplace=True)
        lagged.set_index(merge_columns, inplace=True)
        
        df = pd.merge(df, lagged,on=merge_columns,how='left',copy=False)
        gc.collect()
        df.reset_index(inplace=True)
    
        df = downcast_lags(df, lagged_names)
        del lagged
        gc.collect()
        
    return df

In [11]:
gc.collect()
def lagged_name(lag_column, lag):
    return "%s_lag_%d" % (lag_column, lag)

lag_columns = [
 'item_block_units',
 'item_block_turnover',
 'item_mean_price_block',
 ]

idx_columns = ['item_id']

training = add_lag_features(training,lag_columns,idx_columns)

1
2
3


In [13]:

pd.set_option('display.max_columns', None)  
pd.set_option('display.expand_frame_repr', False)
pd.set_option('max_colwidth', -1)
training[(training['item_id'] == 30) & (training['shop_id'] == 30)]\
                .drop_duplicates(['item_id', 'date_block_num'])[['item_id','shop_id','date_block_num','item_block_units','item_block_turnover',\
                'item_block_units_lag_1',
 'item_block_turnover_lag_1', 'item_mean_price_block_lag_1',
 'item_block_units_lag_2', 'item_block_turnover_lag_2',
 'item_mean_price_block_lag_2', 'item_block_units_lag_3',
 'item_block_turnover_lag_3' ,'item_mean_price_block_lag_3'
                                                                ]]

,item_id,shop_id,date_block_num,item_block_units,item_block_turnover,item_block_units_lag_1,item_block_turnover_lag_1,item_mean_price_block_lag_1,item_block_units_lag_2,item_block_turnover_lag_2,item_mean_price_block_lag_2,item_block_units_lag_3,item_block_turnover_lag_3,item_mean_price_block_lag_3
1210,30,30,12,58,9802,0,0,0.000000,0,0,0.000000,0,0,0.000000
1211,30,30,13,24,3986,58,9802,169.000000,0,0,0.000000,0,0,0.000000
1212,30,30,14,31,5239,24,3986,166.083328,58,9802,169.000000,0,0,0.000000
1213,30,30,15,21,3479,31,5239,169.000000,24,3986,166.083328,58,9802,169.000000
1214,30,30,16,16,2634,21,3479,165.666672,31,5239,169.000000,24,3986,166.083328
1215,30,30,17,13,2197,16,2634,164.625000,21,3479,165.666672,31,5239,169.000000
1216,30,30,18,13,2127,13,2197,169.000000,16,2634,164.625000,21,3479,165.666672
1217,30,30,19,12,2028,13,2127,163.615387,13,2197,169.000000,16,2634,164.625000
1218,30,30,20,11,1859,12,2028,169.000000,13,2127,163.615387,13,2197,169.000000
1219,30,30,21,13,2197,11,1859,169.000000,12,2028,169.000000,13,2127,163.615387


In [15]:
gc.collect()
def lagged_name(lag_column, lag):
    return "%s_lag_%d" % (lag_column, lag)

lag_columns = [
 'category_block_units',
 'category_block_turnover',
 'category_mean_price_block',
 'subcategory_block_units',
 'subcategory_block_turnover',
 'subcategory_mean_price_block',
 ]

idx_columns = ['item_category_id']

training = add_lag_features(training,lag_columns,idx_columns)

1
2
3


In [17]:
gc.collect()
def lagged_name(lag_column, lag):
    return "%s_lag_%d" % (lag_column, lag)

lag_columns = [
 'shop_block_units',
 'shop_block_turnover',
 'shop_mean_price_block',
 'area_block_units',
 'area_block_turnover',
 'area_mean_price_block'
 ]

idx_columns = ['shop_id']


training = add_lag_features(training,lag_columns,idx_columns)

1
2
3


In [3]:
#gc.collect()
#training.to_pickle("pickled/training_mid_lags")
#training = pd.read_pickle("pickled/training_mid_lags")

In [7]:
lag_columns = [
  'shop_category_block_units',
 'shop_category_turnover',
 'shop_category_mean_price_block'
 ]

idx_columns = ['shop_id','item_category_id']


training = add_lag_features(training,lag_columns,idx_columns)

1
2
3


In [8]:
training.drop(columns=['lagged_block'],inplace=True)

In [9]:
training.columns.values

array(['shop_id', 'item_category_id', 'item_id', 'date_block_num',
       'month', 'year', 'item_first_block', 'item_last_block',
       'is_first_two_blocks', 'is_last_two_blocks', 'item_units',
       'item_mean_units_block', 'item_day_units', 'item_mean_units_day',
       'item_max_units_block', 'item_min_units_block',
       'item_max_units_day', 'item_min_units_day', 'item_turnover',
       'item_mean_turnover_block', 'item_day_turnover',
       'item_mean_turnover_day', 'item_max_turnover_block',
       'item_min_turnover_block', 'item_max_turnover_day',
       'item_min_turnover_day', 'item_days_of_activity',
       'item_blocks_of_activity', 'item_days_since_start',
       'item_mean_day_between_activity',
       'item_longest_stretch_days_without_activity',
       'item_longest_stretch_blocks_without_activity',
       'item_longest_stretch_block_with_activity',
       'item_number_of_consecutive_days_with_activity',
       'item_days_between_start_and_first_activity',
       '

In [3]:
gc.collect()
#training.to_pickle("pickled/training_post_lags")
training = pd.read_pickle("pickled/training_post_lags")

In [4]:
gc.collect()
training.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8333930 entries, 0 to 8333929
Columns: 209 entries, item_id to shop_category_mean_price_block
dtypes: bool(5), category(2), float32(79), int16(23), int32(29), int8(2), object(16), uint16(24), uint32(17), uint8(12)
memory usage: 8.9 GB


In [18]:
training.dtypes

shop_id                                  uint64
item_category_id                         uint64
item_id                                   int64
date_block_num                            uint8
month                                     uint8
year                                     uint16
item_first_block                          uint8
item_last_block                           uint8
is_first_two_blocks                      object
is_last_two_blocks                       object
item_units                              float32
item_mean_units_block                   float32
item_day_units                            int16
item_mean_units_day                     float32
item_max_units_block                      int16
item_min_units_block                      int16
item_max_units_day                        int16
item_min_units_day                         int8
item_turnover                             int32
item_mean_turnover_block                float32
item_day_turnover                       

In [11]:
training.reset_index(inplace=True)

In [12]:
cols = ['shop_id','item_id', 'date_block_num']
training.set_index(cols, inplace=True)
transactions.drop_duplicates(cols, inplace=True)
transactions.set_index(cols, inplace=True)

training = pd.merge(training, transactions['y'], on=cols, how='left', copy=False)

training.reset_index(inplace=True)

In [13]:
training['y'] = training['y'].fillna(0)

In [39]:
gc.collect()
#training.to_pickle("pickled/training_pre_catboost")
training = pd.read_pickle("pickled/training_pre_catboost")

FileNotFoundError: [Errno 2] No such file or directory: 'pickled/training_pre_catboost'

In [9]:
gc.collect()

0

In [10]:
pd.set_option('display.max_rows', 300)
for col in training.columns:
    if training.dtypes[col].kind == 'b':
        print(col, training.dtypes[col])

shop_TC bool
shop_TRK bool
shop_SEC bool
shop_shopping_center bool
shop_moscow bool


In [11]:
for m in range(1,13):
    training[str(m)] = training['month'] == m

In [12]:
x_train = training[training['date_block_num'] < 33]
y_train = x_train['y']
#x_train = x_train.drop(columns=['y'])

x_val = training[training['date_block_num'] == 33]
y_val = x_val['y']
#x_val = x_val.drop(columns=['y'])


In [13]:
del training
gc.collect()

84

In [16]:
#x_train.to_pickle("pickled/x_train")
x_train = pd.read_pickle("pickled/x_train")
#y_train.to_pickle("pickled/y_train")
y_train = pd.read_pickle("pickled/y_train")
#x_val.to_pickle("pickled/x_val")
x_val = pd.read_pickle("pickled/x_val")
#y_val.to_pickle("pickled/y_val")
y_val = pd.read_pickle("pickled/y_val")

In [19]:
x_train.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2000000 entries, 5883565 to 167468
Columns: 276 entries, shop_id to 12
dtypes: bool(17), category(2), float32(97), float64(1), int16(22), int32(29), int64(25), int8(2), object(16), uint16(30), uint32(23), uint64(3), uint8(9)
memory usage: 2.8 GB


In [10]:
pos_train_len = len(y_train[y_train != 0])
pos_train_len

887869

In [16]:
zeros_keep_indices_train = y_train[y_train == 0].sample(int(pos_train_len/4)).index
non_zeros_train_indices = y_train[y_train != 0].index
train_indices = np.append(np.array(zeros_keep_indices_train), np.array(non_zeros_train_indices))

x_train = x_train.loc[train_indices]
#x_train.drop_duplicates(['shop_id','item_id', 'date_block_num'], inplace=True)
y_train = y_train.loc[x_train.index]

In [15]:
len(non_zeros_train_indices)

887869

In [34]:
len(x_train)

7955115

In [17]:
pos_val_len = len(y_val[y_val != 0])
pos_val_len

31471

In [18]:
pos_val_len = len(y_val[y_val != 0])
zeros_keep_indices_val = y_val[y_val == 0].sample(int(pos_val_len/4)).index
non_zeros_val_indices = (y_val != 0).index
val_indices = np.append(np.array(zeros_keep_indices_val), np.array(non_zeros_val_indices))


y_val = y_val.loc[val_indices]
x_val = x_val.loc[val_indices]

In [33]:
len(x_val)

386682

In [36]:
training.columns.values

array(['shop_id', 'item_id', 'date_block_num', 'item_category_id',
       'month', 'year', 'item_first_block', 'item_last_block',
       'is_first_two_blocks', 'is_last_two_blocks', 'item_units',
       'item_mean_units_block', 'item_day_units', 'item_mean_units_day',
       'item_max_units_block', 'item_min_units_block',
       'item_max_units_day', 'item_min_units_day', 'item_turnover',
       'item_mean_turnover_block', 'item_day_turnover',
       'item_mean_turnover_day', 'item_max_turnover_block',
       'item_min_turnover_block', 'item_max_turnover_day',
       'item_min_turnover_day', 'item_days_of_activity',
       'item_blocks_of_activity', 'item_days_since_start',
       'item_mean_day_between_activity',
       'item_longest_stretch_days_without_activity',
       'item_longest_stretch_blocks_without_activity',
       'item_longest_stretch_block_with_activity',
       'item_number_of_consecutive_days_with_activity',
       'item_days_between_start_and_first_activity',
       '

In [5]:
cb_features = [
       'item_first_block', 'item_last_block',
       'is_first_two_blocks', 'is_last_two_blocks', 'item_units',
       'item_mean_units_block', 'item_mean_units_day',
       'item_max_units_block', 'item_min_units_block',
       'item_max_units_day', 'item_min_units_day', 'item_turnover',
       'item_mean_turnover_block',
       'item_mean_turnover_day', 'item_max_turnover_block',
       'item_min_turnover_block', 'item_max_turnover_day',
       'item_min_turnover_day', 'item_days_of_activity',
       'item_blocks_of_activity', 'item_days_since_start',
       'item_mean_day_between_activity',
       'item_longest_stretch_days_without_activity',
       'item_longest_stretch_blocks_without_activity',
       'item_longest_stretch_block_with_activity',
       'item_number_of_consecutive_days_with_activity',
       'item_days_between_start_and_first_activity',
       'item_blocks_between_start_and_first_activity', 'item_first_day',
       'item_last_day', 'item_activity_on_all_blocks', 'item_mean_price',
       'item_min_price', 'item_max_price', 'item_number_different_prices',
       'item_price_amplitude', 'item_deviation_mean_category_price',
       'item_first_two_blocks_units', 'item_last_two_blocks_units',
       'item_fluctuation_units_first_last_blocks',
       'item_first_two_blocks_mean_price',
       'item_last_two_blocks_mean_price',
       'item_fluctuation_price_first_last_blocks',
       'item_share_of_total_units', 'item_share_of_total_turnover',
       'category_units', 'category_mean_units_block',
       'category_mean_units_day',
       'category_max_units_block', 'category_min_units_block',
       'category_max_units_day', 'category_min_units_day',
       'category_turnover', 'category_mean_turnover_block',
       'category_mean_turnover_day',
       'category_max_turnover_block', 'category_min_turnover_block',
       'category_max_turnover_day', 'category_min_turnover_day',
       'category_mean_price', 'category_min_price', 'category_max_price',
       'category_first_two_blocks_units',
       'category_last_two_blocks_units',
       'category_fluctuation_units_first_last_blocks',
       'category_first_two_blocks_mean_price',
       'category_last_two_blocks_mean_price',
       'category_fluctuation_price_first_last_blocks',
       'video_game', 'gaming_old_gen', 'gaming_new_gen', 'pc_games',
       'payment_cards', 'movies', 'movies_niche', 'books', 'music',
       'music_CD', 'music_vinyl', 'gifts', 'software',
       'subcategory_units', 'subcategory_mean_units_block',
       'subcategory_mean_units_day',
       'subcategory_max_units_block', 'subcategory_min_units_block',
       'subcategory_max_units_day', 'subcategory_min_units_day',
       'subcategory_turnover', 'subcategory_mean_turnover_block',
       'subcategory_mean_turnover_day',
       'subcategory_max_turnover_block', 'subcategory_min_turnover_block',
       'subcategory_max_turnover_day', 'subcategory_min_turnover_day',
       'category_share_of_total_units',
       'category_share_of_total_turnover',
       'subcategory_share_of_total_units',
       'subcategory_share_of_total_turnover',
       'subcategory_first_two_blocks_units',
       'subcategory_last_two_blocks_units',
       'subcategory_fluctuation_units_first_last_blocks',
       'subcategory_first_two_blocks_mean_price',
       'subcategory_last_two_blocks_mean_price',
       'subcategory_fluctuation_price_first_last_blocks',
       'shop_units', 'shop_mean_units_block', 
       'shop_mean_units_day', 'shop_max_units_block',
       'shop_min_units_block', 'shop_max_units_day', 'shop_min_units_day',
       'shop_turnover', 'shop_mean_turnover_block', 
       'shop_mean_turnover_day', 'shop_max_turnover_block',
       'shop_min_turnover_block', 'shop_max_turnover_day',
       'shop_min_turnover_day', 'shop_mean_price',
       'shop_first_two_blocks_units', 'shop_last_two_blocks_units',
       'shop_fluctuation_units_first_last_blocks',
       'shop_first_two_blocks_mean_price',
       'shop_last_two_blocks_mean_price',
       'shop_fluctuation_price_first_last_blocks', 'shop_share_of_units',
       'shop_share_of_turnover', 'shop_TC', 'shop_TRK', 'shop_SEC',
       'shop_shopping_center', 'shop_moscow', 'max_category_units',
       'max_category_turnover', 'area_units',
       'area_mean_units_block', 'area_mean_units_day',
       'area_max_units_block', 'area_min_units_block',
       'area_max_units_day', 'area_min_units_day', 'area_turnover',
       'area_mean_turnover_block',
       'area_mean_turnover_day', 'area_max_turnover_block',
       'area_min_turnover_block', 'area_max_turnover_day',
       'area_min_turnover_day', 'area_mean_price',
       'area_first_two_blocks_units', 'area_last_two_blocks_units',
       'area_fluctuation_units_first_last_blocks',
       'area_first_two_blocks_mean_price',
       'area_last_two_blocks_mean_price',
       'area_fluctuation_price_first_last_blocks',
       'shop_category_units', 'shop_category_mean_units_block',
       'shop_category_mean_units_day',
       'shop_category_max_units_block', 'shop_category_min_units_block',
       'shop_category_max_units_day', 'shop_category_min_units_day',
       'shop_category_turnover', 'shop_category_mean_turnover_block',
       'shop_category_mean_turnover_day',
       'shop_category_max_turnover_block',
       'shop_category_min_turnover_block',
       'shop_category_max_turnover_day', 'shop_category_min_turnover_day',
       'shop_category_mean_price',
       'item_block_units_lag_1', 'item_block_turnover_lag_1',
       'item_mean_price_block_lag_1', 'item_block_units_lag_2',
       'item_block_turnover_lag_2', 'item_mean_price_block_lag_2',
       'item_block_units_lag_3', 'item_block_turnover_lag_3',
       'item_mean_price_block_lag_3', 'category_block_units_lag_1',
       'category_block_turnover_lag_1', 'category_mean_price_block_lag_1',
       'subcategory_block_units_lag_1',
       'subcategory_block_turnover_lag_1',
       'subcategory_mean_price_block_lag_1', 'category_block_units_lag_2',
       'category_block_turnover_lag_2', 'category_mean_price_block_lag_2',
       'subcategory_block_units_lag_2',
       'subcategory_block_turnover_lag_2',
       'subcategory_mean_price_block_lag_2', 'category_block_units_lag_3',
       'category_block_turnover_lag_3', 'category_mean_price_block_lag_3',
       'subcategory_block_units_lag_3',
       'subcategory_block_turnover_lag_3',
       'subcategory_mean_price_block_lag_3', 'shop_block_units_lag_1',
       'shop_block_turnover_lag_1', 'shop_mean_price_block_lag_1',
       'area_block_units_lag_1', 'area_block_turnover_lag_1',
       'area_mean_price_block_lag_1', 'shop_block_units_lag_2',
       'shop_block_turnover_lag_2', 'shop_mean_price_block_lag_2',
       'area_block_units_lag_2', 'area_block_turnover_lag_2',
       'area_mean_price_block_lag_2', 'shop_block_units_lag_3',
       'shop_block_turnover_lag_3', 'shop_mean_price_block_lag_3',
       'area_block_units_lag_3', 'area_block_turnover_lag_3',
       'area_mean_price_block_lag_3',
       '1', '2', '3', '4', '5', '6','7', '8', '9', '10', '11', '12']


In [36]:
cb_model = CatBoostRegressor(iterations=70000,
                             learning_rate=0.001,
                             eval_metric='RMSE',
                             #thread_count=16,
                             task_type = "GPU",
                             use_best_model=True,
                             #l2_leaf_reg = 1000,
                             od_type = "Iter",
                             od_wait = 30,
                             #random_strength = 10,
                             #bagging_temperature = 1,
                             #one_hot_max_size = 2,
                             random_seed = 42)

#drops = ['subcategory','area']
#x_train = x_train.drop(columns=drops)
#x_val = x_val.drop(columns=drops)


cb_model.fit(x_train[cb_features], y_train, #cat_features=categorical_features_indices,
             eval_set=(x_val[cb_features],y_val),
             #cat_features=categorical_features_pos,         
             verbose=True)

0:	learn: 1.0781136	test: 0.9142845	best: 0.9142845 (0)	total: 108ms	remaining: 2h 5m 54s
1:	learn: 1.0777921	test: 0.9140996	best: 0.9140996 (1)	total: 202ms	remaining: 1h 57m 36s
2:	learn: 1.0774676	test: 0.9139167	best: 0.9139167 (2)	total: 295ms	remaining: 1h 54m 46s
3:	learn: 1.0771447	test: 0.9137301	best: 0.9137301 (3)	total: 385ms	remaining: 1h 52m 15s
4:	learn: 1.0768218	test: 0.9135452	best: 0.9135452 (4)	total: 475ms	remaining: 1h 50m 42s
5:	learn: 1.0764999	test: 0.9133587	best: 0.9133587 (5)	total: 563ms	remaining: 1h 49m 31s
6:	learn: 1.0761791	test: 0.9131781	best: 0.9131781 (6)	total: 654ms	remaining: 1h 48m 54s
7:	learn: 1.0758570	test: 0.9129922	best: 0.9129922 (7)	total: 740ms	remaining: 1h 47m 52s
8:	learn: 1.0755377	test: 0.9128068	best: 0.9128068 (8)	total: 829ms	remaining: 1h 47m 30s
9:	learn: 1.0752194	test: 0.9126220	best: 0.9126220 (9)	total: 918ms	remaining: 1h 47m 6s
10:	learn: 1.0749000	test: 0.9124485	best: 0.9124485 (10)	total: 1.01s	remaining: 1h 47m 14s

90:	learn: 1.0508532	test: 0.8989970	best: 0.8989970 (90)	total: 8.37s	remaining: 1h 47m 8s
91:	learn: 1.0505649	test: 0.8988018	best: 0.8988018 (91)	total: 8.46s	remaining: 1h 47m 5s
92:	learn: 1.0502832	test: 0.8986667	best: 0.8986667 (92)	total: 8.54s	remaining: 1h 47m 3s
93:	learn: 1.0500031	test: 0.8985121	best: 0.8985121 (93)	total: 8.64s	remaining: 1h 47m 4s
94:	learn: 1.0497204	test: 0.8983555	best: 0.8983555 (94)	total: 8.73s	remaining: 1h 47m 3s
95:	learn: 1.0494385	test: 0.8982115	best: 0.8982115 (95)	total: 8.82s	remaining: 1h 47m 6s
96:	learn: 1.0491600	test: 0.8980586	best: 0.8980586 (96)	total: 8.92s	remaining: 1h 47m 7s
97:	learn: 1.0488807	test: 0.8979110	best: 0.8979110 (97)	total: 9.01s	remaining: 1h 47m 8s
98:	learn: 1.0486025	test: 0.8977558	best: 0.8977558 (98)	total: 9.11s	remaining: 1h 47m 10s
99:	learn: 1.0483195	test: 0.8976096	best: 0.8976096 (99)	total: 9.2s	remaining: 1h 47m 10s
100:	learn: 1.0480384	test: 0.8974666	best: 0.8974666 (100)	total: 9.29s	remain

177:	learn: 1.0276120	test: 0.8860299	best: 0.8860299 (177)	total: 16.3s	remaining: 1h 46m 36s
178:	learn: 1.0273571	test: 0.8858596	best: 0.8858596 (178)	total: 16.4s	remaining: 1h 46m 37s
179:	learn: 1.0271042	test: 0.8857313	best: 0.8857313 (179)	total: 16.5s	remaining: 1h 46m 38s
180:	learn: 1.0268530	test: 0.8855922	best: 0.8855922 (180)	total: 16.6s	remaining: 1h 46m 35s
181:	learn: 1.0266043	test: 0.8854697	best: 0.8854697 (181)	total: 16.7s	remaining: 1h 46m 34s
182:	learn: 1.0263529	test: 0.8853420	best: 0.8853420 (182)	total: 16.8s	remaining: 1h 46m 34s
183:	learn: 1.0261008	test: 0.8852051	best: 0.8852051 (183)	total: 16.8s	remaining: 1h 46m 32s
184:	learn: 1.0258532	test: 0.8850829	best: 0.8850829 (184)	total: 16.9s	remaining: 1h 46m 32s
185:	learn: 1.0256048	test: 0.8849473	best: 0.8849473 (185)	total: 17s	remaining: 1h 46m 37s
186:	learn: 1.0253607	test: 0.8848319	best: 0.8848319 (186)	total: 17.1s	remaining: 1h 46m 39s
187:	learn: 1.0251147	test: 0.8847181	best: 0.884718

264:	learn: 1.0070584	test: 0.8748710	best: 0.8748710 (264)	total: 24.3s	remaining: 1h 46m 35s
265:	learn: 1.0068355	test: 0.8747514	best: 0.8747514 (265)	total: 24.4s	remaining: 1h 46m 33s
266:	learn: 1.0066078	test: 0.8746032	best: 0.8746032 (266)	total: 24.5s	remaining: 1h 46m 33s
267:	learn: 1.0063911	test: 0.8744995	best: 0.8744995 (267)	total: 24.6s	remaining: 1h 46m 34s
268:	learn: 1.0061723	test: 0.8743986	best: 0.8743986 (268)	total: 24.7s	remaining: 1h 46m 34s
269:	learn: 1.0059555	test: 0.8742859	best: 0.8742859 (269)	total: 24.8s	remaining: 1h 46m 35s
270:	learn: 1.0057372	test: 0.8741560	best: 0.8741560 (270)	total: 24.9s	remaining: 1h 46m 39s
271:	learn: 1.0055159	test: 0.8740474	best: 0.8740474 (271)	total: 25s	remaining: 1h 46m 40s
272:	learn: 1.0052956	test: 0.8739301	best: 0.8739301 (272)	total: 25.1s	remaining: 1h 46m 41s
273:	learn: 1.0050735	test: 0.8738047	best: 0.8738047 (273)	total: 25.2s	remaining: 1h 46m 45s
274:	learn: 1.0048500	test: 0.8736565	best: 0.873656

351:	learn: 0.9888730	test: 0.8652040	best: 0.8652040 (351)	total: 32.8s	remaining: 1h 48m 16s
352:	learn: 0.9886807	test: 0.8651174	best: 0.8651174 (352)	total: 32.9s	remaining: 1h 48m 18s
353:	learn: 0.9884901	test: 0.8650120	best: 0.8650120 (353)	total: 33s	remaining: 1h 48m 19s
354:	learn: 0.9882960	test: 0.8649268	best: 0.8649268 (354)	total: 33.1s	remaining: 1h 48m 19s
355:	learn: 0.9881010	test: 0.8648180	best: 0.8648180 (355)	total: 33.2s	remaining: 1h 48m 22s
356:	learn: 0.9879048	test: 0.8647087	best: 0.8647087 (356)	total: 33.3s	remaining: 1h 48m 22s
357:	learn: 0.9877116	test: 0.8646018	best: 0.8646018 (357)	total: 33.4s	remaining: 1h 48m 23s
358:	learn: 0.9875151	test: 0.8645008	best: 0.8645008 (358)	total: 33.5s	remaining: 1h 48m 23s
359:	learn: 0.9873217	test: 0.8644029	best: 0.8644029 (359)	total: 33.6s	remaining: 1h 48m 26s
360:	learn: 0.9871298	test: 0.8643165	best: 0.8643165 (360)	total: 33.7s	remaining: 1h 48m 27s
361:	learn: 0.9869350	test: 0.8642123	best: 0.864212

...

4195:	learn: 0.8104784	test: 0.7790176	best: 0.7790176 (4195)	total: 7m 22s	remaining: 1h 55m 38s
4196:	learn: 0.8104633	test: 0.7790193	best: 0.7790176 (4195)	total: 7m 22s	remaining: 1h 55m 38s
4197:	learn: 0.8104501	test: 0.7790189	best: 0.7790176 (4195)	total: 7m 22s	remaining: 1h 55m 38s
4198:	learn: 0.8104409	test: 0.7790113	best: 0.7790113 (4198)	total: 7m 22s	remaining: 1h 55m 38s
4199:	learn: 0.8104292	test: 0.7790040	best: 0.7790040 (4199)	total: 7m 22s	remaining: 1h 55m 38s
4200:	learn: 0.8104163	test: 0.7790031	best: 0.7790031 (4200)	total: 7m 22s	remaining: 1h 55m 38s
4201:	learn: 0.8104063	test: 0.7790040	best: 0.7790031 (4200)	total: 7m 23s	remaining: 1h 55m 38s
4202:	learn: 0.8103965	test: 0.7789992	best: 0.7789992 (4202)	total: 7m 23s	remaining: 1h 55m 38s
4203:	learn: 0.8103876	test: 0.7789992	best: 0.7789992 (4202)	total: 7m 23s	remaining: 1h 55m 38s
4204:	learn: 0.8103785	test: 0.7789955	best: 0.7789955 (4204)	total: 7m 23s	remaining: 1h 55m 38s
4205:	learn: 0.81035

4280:	learn: 0.8095312	test: 0.7786855	best: 0.7786855 (4280)	total: 7m 31s	remaining: 1h 55m 38s
4281:	learn: 0.8095225	test: 0.7786819	best: 0.7786819 (4281)	total: 7m 32s	remaining: 1h 55m 38s
4282:	learn: 0.8095091	test: 0.7786814	best: 0.7786814 (4282)	total: 7m 32s	remaining: 1h 55m 38s
4283:	learn: 0.8094990	test: 0.7786757	best: 0.7786757 (4283)	total: 7m 32s	remaining: 1h 55m 38s
4284:	learn: 0.8094880	test: 0.7786748	best: 0.7786748 (4284)	total: 7m 32s	remaining: 1h 55m 38s
4285:	learn: 0.8094789	test: 0.7786702	best: 0.7786702 (4285)	total: 7m 32s	remaining: 1h 55m 38s
4286:	learn: 0.8094667	test: 0.7786693	best: 0.7786693 (4286)	total: 7m 32s	remaining: 1h 55m 38s
4287:	learn: 0.8094603	test: 0.7786650	best: 0.7786650 (4287)	total: 7m 32s	remaining: 1h 55m 38s
4288:	learn: 0.8094560	test: 0.7786632	best: 0.7786632 (4288)	total: 7m 32s	remaining: 1h 55m 38s
4289:	learn: 0.8094493	test: 0.7786602	best: 0.7786602 (4289)	total: 7m 32s	remaining: 1h 55m 37s
4290:	learn: 0.80943

4365:	learn: 0.8086152	test: 0.7784386	best: 0.7784352 (4362)	total: 7m 41s	remaining: 1h 55m 38s
4366:	learn: 0.8086008	test: 0.7784410	best: 0.7784352 (4362)	total: 7m 41s	remaining: 1h 55m 38s
4367:	learn: 0.8085873	test: 0.7784425	best: 0.7784352 (4362)	total: 7m 41s	remaining: 1h 55m 38s
4368:	learn: 0.8085729	test: 0.7784391	best: 0.7784352 (4362)	total: 7m 41s	remaining: 1h 55m 38s
4369:	learn: 0.8085611	test: 0.7784391	best: 0.7784352 (4362)	total: 7m 42s	remaining: 1h 55m 38s
4370:	learn: 0.8085494	test: 0.7783676	best: 0.7783676 (4370)	total: 7m 42s	remaining: 1h 55m 38s
4371:	learn: 0.8085452	test: 0.7783644	best: 0.7783644 (4371)	total: 7m 42s	remaining: 1h 55m 38s
4372:	learn: 0.8085313	test: 0.7783577	best: 0.7783577 (4372)	total: 7m 42s	remaining: 1h 55m 38s
4373:	learn: 0.8085224	test: 0.7783652	best: 0.7783577 (4372)	total: 7m 42s	remaining: 1h 55m 38s
4374:	learn: 0.8085100	test: 0.7783586	best: 0.7783577 (4372)	total: 7m 42s	remaining: 1h 55m 38s
4375:	learn: 0.80850

4449:	learn: 0.8077353	test: 0.7781782	best: 0.7781758 (4448)	total: 7m 50s	remaining: 1h 55m 37s
4450:	learn: 0.8077209	test: 0.7781724	best: 0.7781724 (4450)	total: 7m 51s	remaining: 1h 55m 37s
4451:	learn: 0.8077149	test: 0.7781727	best: 0.7781724 (4450)	total: 7m 51s	remaining: 1h 55m 37s
4452:	learn: 0.8077002	test: 0.7781741	best: 0.7781724 (4450)	total: 7m 51s	remaining: 1h 55m 37s
4453:	learn: 0.8076869	test: 0.7781718	best: 0.7781718 (4453)	total: 7m 51s	remaining: 1h 55m 37s
4454:	learn: 0.8076742	test: 0.7781672	best: 0.7781672 (4454)	total: 7m 51s	remaining: 1h 55m 37s
4455:	learn: 0.8076659	test: 0.7781603	best: 0.7781603 (4455)	total: 7m 51s	remaining: 1h 55m 38s
4456:	learn: 0.8076530	test: 0.7781609	best: 0.7781603 (4455)	total: 7m 51s	remaining: 1h 55m 38s
4457:	learn: 0.8076421	test: 0.7781561	best: 0.7781561 (4457)	total: 7m 51s	remaining: 1h 55m 38s
4458:	learn: 0.8076293	test: 0.7781596	best: 0.7781561 (4457)	total: 7m 52s	remaining: 1h 55m 38s
4459:	learn: 0.80761

4534:	learn: 0.8068342	test: 0.7779503	best: 0.7779431 (4527)	total: 8m	remaining: 1h 55m 38s
4535:	learn: 0.8068236	test: 0.7779457	best: 0.7779431 (4527)	total: 8m	remaining: 1h 55m 38s
4536:	learn: 0.8068103	test: 0.7779491	best: 0.7779431 (4527)	total: 8m	remaining: 1h 55m 38s
4537:	learn: 0.8067977	test: 0.7779438	best: 0.7779431 (4527)	total: 8m 1s	remaining: 1h 55m 38s
4538:	learn: 0.8067881	test: 0.7779448	best: 0.7779431 (4527)	total: 8m 1s	remaining: 1h 55m 38s
4539:	learn: 0.8067795	test: 0.7779348	best: 0.7779348 (4539)	total: 8m 1s	remaining: 1h 55m 38s
4540:	learn: 0.8067733	test: 0.7779337	best: 0.7779337 (4540)	total: 8m 1s	remaining: 1h 55m 38s
4541:	learn: 0.8067652	test: 0.7779256	best: 0.7779256 (4541)	total: 8m 1s	remaining: 1h 55m 38s
4542:	learn: 0.8067554	test: 0.7779209	best: 0.7779209 (4542)	total: 8m 1s	remaining: 1h 55m 38s
4543:	learn: 0.8067443	test: 0.7779228	best: 0.7779209 (4542)	total: 8m 1s	remaining: 1h 55m 38s
4544:	learn: 0.8067337	test: 0.7779207	

In [3]:
#training.to_pickle("pickled/training")
#training = pd.read_pickle("pickled/training")

#pickle.dump(cb_model, open( "pickled/cb_model", "wb"), protocol=4)

cb_model = pickle.load( open( "pickled/cb_model", "rb" ) )

In [6]:
scores = {}
for i,score in enumerate(cb_model.get_feature_importance()):
    scores[cb_features[i]] = score

In [15]:
best_cb_features = [item[0] for item in scores.items() if item[1] > 0.1]

In [10]:
sorted(scores.items(), key=lambda x: x[1])[::-1]

[('item_block_units_lag_1', 32.740321314769616),
 ('shop_category_units', 7.3037773859846205),
 ('item_share_of_total_units', 4.906273669850913),
 ('shop_first_two_blocks_units', 4.547441700039596),
 ('item_units', 4.53217847313849),
 ('item_mean_price_block_lag_2', 3.5639239412024306),
 ('item_number_of_consecutive_days_with_activity', 3.514854655108098),
 ('item_first_two_blocks_units', 3.287811617438042),
 ('shop_max_turnover_block', 3.215991046364685),
 ('shop_category_turnover', 2.1883809310526185),
 ('shop_max_units_block', 2.1503020232523826),
 ('shop_share_of_units', 1.6670817918624485),
 ('shop_units', 1.4990774111978884),
 ('shop_block_units_lag_1', 1.4677487535371527),
 ('item_last_two_blocks_units', 1.4319509614392278),
 ('item_mean_units_block', 1.4086443066998504),
 ('category_mean_turnover_day', 1.304919157401533),
 ('shop_block_turnover_lag_1', 1.1531907893360216),
 ('12', 1.0944565263354495),
 ('shop_min_turnover_block', 1.0516317566694033),
 ('item_first_day', 1.01175

In [27]:
cb_features = xg_features

In [19]:
xg_features = best_cb_features

gc.collect()
params =   {
    'objective' : 'gpu:reg:linear',
    'tree_method':'gpu_hist',
    #'gpu_id': 0,
    'learning_rate': 0.001, 
    #'gamma' : 0.3, 
    #'min_child_weight' : 3,
    #'nthread' : 16,
    #'max_depth' : 30,
    #'subsample' : 0.9, 
    #'colsample_bytree' : 0.5, 
    'seed':42, 
    'eval_metric' : "rmse",
    'num_boost_round' : 70000,
    #'n_estimators':999,
    #'max_leaves': 300
}


tr_data = xgb.DMatrix(x_train[xg_features], y_train)
va_data = xgb.DMatrix(x_val[xg_features], y_val)


watchlist = [(tr_data, 'train'), (va_data, 'valid')]

xg_model = xgb.train(params, tr_data, 70000, watchlist, maximize=False, early_stopping_rounds = 30, verbose_eval=True)

[13:12:28] /workspace/src/objective/regression_obj.cu:153: gpu:reg:linear is now deprecated, use reg:linear instead.
[0]	train-rmse:1.0897	valid-rmse:0.962499
Multiple eval metrics have been passed: 'valid-rmse' will be used for early stopping.

Will train until valid-rmse hasn't improved in 30 rounds.
[1]	train-rmse:1.08928	valid-rmse:0.962189
[2]	train-rmse:1.08886	valid-rmse:0.961879
[3]	train-rmse:1.08845	valid-rmse:0.961571
[4]	train-rmse:1.08803	valid-rmse:0.961262
[5]	train-rmse:1.08761	valid-rmse:0.960954
[6]	train-rmse:1.0872	valid-rmse:0.960647
[7]	train-rmse:1.08679	valid-rmse:0.960341
[8]	train-rmse:1.08637	valid-rmse:0.960035
[9]	train-rmse:1.08596	valid-rmse:0.95973
[10]	train-rmse:1.08555	valid-rmse:0.959424
[11]	train-rmse:1.08514	valid-rmse:0.95912
[12]	train-rmse:1.08473	valid-rmse:0.958816
[13]	train-rmse:1.08432	valid-rmse:0.958513
[14]	train-rmse:1.08391	valid-rmse:0.958209
[15]	train-rmse:1.0835	valid-rmse:0.957907
[16]	train-rmse:1.08309	valid-rmse:0.957604
[17]	

[180]	train-rmse:1.02339	valid-rmse:0.913933
[181]	train-rmse:1.02307	valid-rmse:0.913714
[182]	train-rmse:1.02275	valid-rmse:0.913469
[183]	train-rmse:1.02243	valid-rmse:0.91325
[184]	train-rmse:1.02211	valid-rmse:0.913016
[185]	train-rmse:1.02179	valid-rmse:0.912796
[186]	train-rmse:1.02147	valid-rmse:0.912553
[187]	train-rmse:1.02116	valid-rmse:0.912336
[188]	train-rmse:1.02084	valid-rmse:0.912101
[189]	train-rmse:1.02053	valid-rmse:0.911883
[190]	train-rmse:1.02021	valid-rmse:0.911652
[191]	train-rmse:1.0199	valid-rmse:0.911438
[192]	train-rmse:1.01957	valid-rmse:0.911204
[193]	train-rmse:1.01926	valid-rmse:0.910989
[194]	train-rmse:1.01895	valid-rmse:0.910751
[195]	train-rmse:1.01863	valid-rmse:0.910539
[196]	train-rmse:1.01832	valid-rmse:0.910325
[197]	train-rmse:1.01801	valid-rmse:0.910094
[198]	train-rmse:1.0177	valid-rmse:0.90988
[199]	train-rmse:1.01738	valid-rmse:0.909644
[200]	train-rmse:1.01707	valid-rmse:0.90943
[201]	train-rmse:1.01676	valid-rmse:0.909197
[202]	train-rms

[361]	train-rmse:0.97249	valid-rmse:0.878608
[362]	train-rmse:0.972242	valid-rmse:0.878428
[363]	train-rmse:0.971997	valid-rmse:0.878267
[364]	train-rmse:0.971747	valid-rmse:0.878087
[365]	train-rmse:0.971502	valid-rmse:0.877918
[366]	train-rmse:0.971253	valid-rmse:0.877759
[367]	train-rmse:0.971004	valid-rmse:0.87758
[368]	train-rmse:0.97076	valid-rmse:0.87741
[369]	train-rmse:0.970513	valid-rmse:0.877228
[370]	train-rmse:0.97027	valid-rmse:0.877061
[371]	train-rmse:0.970028	valid-rmse:0.876902
[372]	train-rmse:0.969781	valid-rmse:0.876725
[373]	train-rmse:0.969539	valid-rmse:0.876556
[374]	train-rmse:0.969294	valid-rmse:0.876381
[375]	train-rmse:0.969052	valid-rmse:0.876214
[376]	train-rmse:0.968806	valid-rmse:0.876059
[377]	train-rmse:0.968562	valid-rmse:0.87588
[378]	train-rmse:0.968321	valid-rmse:0.87572
[379]	train-rmse:0.968078	valid-rmse:0.875546
[380]	train-rmse:0.967839	valid-rmse:0.875388
[381]	train-rmse:0.9676	valid-rmse:0.875223
[382]	train-rmse:0.96736	valid-rmse:0.87504

[541]	train-rmse:0.933296	valid-rmse:0.852148
[542]	train-rmse:0.933111	valid-rmse:0.852025
[543]	train-rmse:0.932926	valid-rmse:0.851901
[544]	train-rmse:0.932739	valid-rmse:0.851777
[545]	train-rmse:0.932549	valid-rmse:0.85165
[546]	train-rmse:0.932361	valid-rmse:0.851526
[547]	train-rmse:0.932177	valid-rmse:0.851403
[548]	train-rmse:0.931995	valid-rmse:0.851288
[549]	train-rmse:0.931807	valid-rmse:0.851169
[550]	train-rmse:0.931623	valid-rmse:0.851054
[551]	train-rmse:0.93144	valid-rmse:0.850935
[552]	train-rmse:0.931258	valid-rmse:0.850813
[553]	train-rmse:0.931073	valid-rmse:0.850693
[554]	train-rmse:0.930885	valid-rmse:0.850563
[555]	train-rmse:0.930705	valid-rmse:0.85045
[556]	train-rmse:0.930519	valid-rmse:0.850331
[557]	train-rmse:0.930337	valid-rmse:0.850211
[558]	train-rmse:0.930153	valid-rmse:0.850092
[559]	train-rmse:0.929972	valid-rmse:0.84998
[560]	train-rmse:0.929789	valid-rmse:0.849854
[561]	train-rmse:0.929605	valid-rmse:0.849744
[562]	train-rmse:0.929419	valid-rmse:0

[720]	train-rmse:0.903477	valid-rmse:0.833034
[721]	train-rmse:0.903331	valid-rmse:0.832936
[722]	train-rmse:0.903187	valid-rmse:0.832843
[723]	train-rmse:0.903042	valid-rmse:0.832743
[724]	train-rmse:0.902898	valid-rmse:0.832644
[725]	train-rmse:0.902756	valid-rmse:0.832554
[726]	train-rmse:0.902613	valid-rmse:0.832466
[727]	train-rmse:0.902469	valid-rmse:0.832364
[728]	train-rmse:0.902326	valid-rmse:0.832268
[729]	train-rmse:0.902183	valid-rmse:0.832175
[730]	train-rmse:0.902036	valid-rmse:0.832086
[731]	train-rmse:0.901893	valid-rmse:0.831992
[732]	train-rmse:0.901751	valid-rmse:0.831905
[733]	train-rmse:0.90161	valid-rmse:0.831807
[734]	train-rmse:0.901468	valid-rmse:0.831711
[735]	train-rmse:0.901322	valid-rmse:0.831623
[736]	train-rmse:0.901179	valid-rmse:0.831528
[737]	train-rmse:0.901038	valid-rmse:0.831433
[738]	train-rmse:0.900894	valid-rmse:0.831344
[739]	train-rmse:0.900754	valid-rmse:0.831257
[740]	train-rmse:0.900614	valid-rmse:0.831167
[741]	train-rmse:0.900475	valid-rms

...

[2484]	train-rmse:0.792371	valid-rmse:0.779795
[2485]	train-rmse:0.792351	valid-rmse:0.779797
[2486]	train-rmse:0.79233	valid-rmse:0.779789
[2487]	train-rmse:0.792304	valid-rmse:0.77977
[2488]	train-rmse:0.792275	valid-rmse:0.779765
[2489]	train-rmse:0.792248	valid-rmse:0.779758
[2490]	train-rmse:0.792227	valid-rmse:0.779752
[2491]	train-rmse:0.792201	valid-rmse:0.779746
[2492]	train-rmse:0.79218	valid-rmse:0.779738
[2493]	train-rmse:0.792153	valid-rmse:0.779729
[2494]	train-rmse:0.792132	valid-rmse:0.779721
[2495]	train-rmse:0.792106	valid-rmse:0.779716
[2496]	train-rmse:0.792075	valid-rmse:0.779706
[2497]	train-rmse:0.792053	valid-rmse:0.779699
[2498]	train-rmse:0.792023	valid-rmse:0.779698
[2499]	train-rmse:0.791997	valid-rmse:0.77968
[2500]	train-rmse:0.791976	valid-rmse:0.779672
[2501]	train-rmse:0.791946	valid-rmse:0.77967
[2502]	train-rmse:0.791925	valid-rmse:0.779663
[2503]	train-rmse:0.791891	valid-rmse:0.779667
[2504]	train-rmse:0.791858	valid-rmse:0.779663
[2505]	train-rmse:

[2660]	train-rmse:0.787973	valid-rmse:0.778613
[2661]	train-rmse:0.787948	valid-rmse:0.778611
[2662]	train-rmse:0.787918	valid-rmse:0.778604
[2663]	train-rmse:0.787894	valid-rmse:0.778592
[2664]	train-rmse:0.787869	valid-rmse:0.778587
[2665]	train-rmse:0.787846	valid-rmse:0.778572
[2666]	train-rmse:0.787829	valid-rmse:0.778565
[2667]	train-rmse:0.787807	valid-rmse:0.77856
[2668]	train-rmse:0.787784	valid-rmse:0.778548
[2669]	train-rmse:0.787763	valid-rmse:0.77854
[2670]	train-rmse:0.787733	valid-rmse:0.778492
[2671]	train-rmse:0.787708	valid-rmse:0.778487
[2672]	train-rmse:0.787684	valid-rmse:0.778472
[2673]	train-rmse:0.787661	valid-rmse:0.778452
[2674]	train-rmse:0.787631	valid-rmse:0.778404
[2675]	train-rmse:0.787615	valid-rmse:0.778397
[2676]	train-rmse:0.787594	valid-rmse:0.778393
[2677]	train-rmse:0.787576	valid-rmse:0.778385
[2678]	train-rmse:0.787552	valid-rmse:0.7784
[2679]	train-rmse:0.787531	valid-rmse:0.778387
[2680]	train-rmse:0.787501	valid-rmse:0.778386
[2681]	train-rmse

[2836]	train-rmse:0.78429	valid-rmse:0.777737
[2837]	train-rmse:0.784271	valid-rmse:0.777723
[2838]	train-rmse:0.784253	valid-rmse:0.777717
[2839]	train-rmse:0.784236	valid-rmse:0.777709
[2840]	train-rmse:0.784219	valid-rmse:0.777702
[2841]	train-rmse:0.784199	valid-rmse:0.777706
[2842]	train-rmse:0.784177	valid-rmse:0.777693
[2843]	train-rmse:0.78416	valid-rmse:0.777685
[2844]	train-rmse:0.784141	valid-rmse:0.777672
[2845]	train-rmse:0.784122	valid-rmse:0.77772
[2846]	train-rmse:0.784102	valid-rmse:0.777723
[2847]	train-rmse:0.784084	valid-rmse:0.777711
[2848]	train-rmse:0.784067	valid-rmse:0.777707
[2849]	train-rmse:0.78405	valid-rmse:0.7777
[2850]	train-rmse:0.784029	valid-rmse:0.777688
[2851]	train-rmse:0.784006	valid-rmse:0.777685
[2852]	train-rmse:0.783986	valid-rmse:0.777681
[2853]	train-rmse:0.783968	valid-rmse:0.777667
[2854]	train-rmse:0.783951	valid-rmse:0.777658
[2855]	train-rmse:0.78393	valid-rmse:0.777644
[2856]	train-rmse:0.783912	valid-rmse:0.777638
[2857]	train-rmse:0.

[3012]	train-rmse:0.781022	valid-rmse:0.77722
[3013]	train-rmse:0.781	valid-rmse:0.777223
[3014]	train-rmse:0.780986	valid-rmse:0.777228
[3015]	train-rmse:0.780969	valid-rmse:0.777209
[3016]	train-rmse:0.780953	valid-rmse:0.777221
[3017]	train-rmse:0.780935	valid-rmse:0.777211
[3018]	train-rmse:0.78092	valid-rmse:0.777211
[3019]	train-rmse:0.780895	valid-rmse:0.777215
[3020]	train-rmse:0.780879	valid-rmse:0.777209
[3021]	train-rmse:0.78086	valid-rmse:0.777211
[3022]	train-rmse:0.780847	valid-rmse:0.777207
[3023]	train-rmse:0.780825	valid-rmse:0.777199
[3024]	train-rmse:0.780809	valid-rmse:0.777199
[3025]	train-rmse:0.780793	valid-rmse:0.777212
[3026]	train-rmse:0.780772	valid-rmse:0.777204
[3027]	train-rmse:0.780758	valid-rmse:0.777217
[3028]	train-rmse:0.780742	valid-rmse:0.777231
[3029]	train-rmse:0.780726	valid-rmse:0.77723
[3030]	train-rmse:0.780711	valid-rmse:0.777222
[3031]	train-rmse:0.78069	valid-rmse:0.777224
[3032]	train-rmse:0.780673	valid-rmse:0.777215
[3033]	train-rmse:0.7

[3188]	train-rmse:0.777954	valid-rmse:0.776458
[3189]	train-rmse:0.777939	valid-rmse:0.776454
[3190]	train-rmse:0.777921	valid-rmse:0.776458
[3191]	train-rmse:0.777907	valid-rmse:0.776453
[3192]	train-rmse:0.777887	valid-rmse:0.776446
[3193]	train-rmse:0.777869	valid-rmse:0.776443
[3194]	train-rmse:0.777851	valid-rmse:0.776446
[3195]	train-rmse:0.777837	valid-rmse:0.776439
[3196]	train-rmse:0.777819	valid-rmse:0.776436
[3197]	train-rmse:0.777795	valid-rmse:0.77643
[3198]	train-rmse:0.777778	valid-rmse:0.776428
[3199]	train-rmse:0.777764	valid-rmse:0.776424
[3200]	train-rmse:0.777744	valid-rmse:0.776429
[3201]	train-rmse:0.777726	valid-rmse:0.776425
[3202]	train-rmse:0.777712	valid-rmse:0.776422
[3203]	train-rmse:0.777695	valid-rmse:0.77642
[3204]	train-rmse:0.777677	valid-rmse:0.776424
[3205]	train-rmse:0.77766	valid-rmse:0.776432
[3206]	train-rmse:0.777644	valid-rmse:0.776418
[3207]	train-rmse:0.77763	valid-rmse:0.776413
[3208]	train-rmse:0.777617	valid-rmse:0.776418
[3209]	train-rmse

In [20]:
pickle.dump(xg_model, open( "pickled/xg_model", "wb"), protocol=4)

#xg_model = pickle.load( open( "pickled/xg_model", "rb" ) )

In [21]:
lg_features = best_cb_features


gc.collect()
lgtrain = lgbm.Dataset(x_train[lg_features], label=y_train)
lgval = lgbm.Dataset(x_val[lg_features], label=y_val)



#[0.00542047893814942, 29, 24, 0.39949465609514856, 1, 0.67943500, 10]
params = {
        "num_threads": 16,
        "device": "gpu",
        "verbosity": -1,
        #"zero_as_missing": "true",
        "boosting":'gbdt',
        "objective" : "regression",
        "metric" : "rmse",
        "seed": 42,
        #"max_bin": 10,#default 255
        #"num_leaves": 10, #default 31
        #"bagging_fraction": 0.7,
        #"bagging_freq": 1,
        #"min_data_in_leaf": 50000,
        #"feature_fraction": 0.7,
        #"lambda_l2": 3,
        #"max_depth": 2,
        #"min_gain_to_split": 10,
        "learning_rate" : 0.001,
        #"histogram_pool_size": 1000,
        #"categorical_column": [0,1,2,3,4]
}

evals_result = {}
model_lgb = lgbm.train(params, lgtrain, 20000, 
                      valid_sets=[lgval], 
                      early_stopping_rounds=100, 
                      verbose_eval=10, 
                      evals_result=evals_result)



Training until validation scores don't improve for 100 rounds.
[10]	valid_0's rmse: 0.900989
[20]	valid_0's rmse: 0.89913
[30]	valid_0's rmse: 0.897319
[40]	valid_0's rmse: 0.895552
[50]	valid_0's rmse: 0.893811
[60]	valid_0's rmse: 0.892121
[70]	valid_0's rmse: 0.890454
[80]	valid_0's rmse: 0.888839
[90]	valid_0's rmse: 0.887264
[100]	valid_0's rmse: 0.885691
[110]	valid_0's rmse: 0.884163
[120]	valid_0's rmse: 0.882629
[130]	valid_0's rmse: 0.88112
[140]	valid_0's rmse: 0.87962
[150]	valid_0's rmse: 0.87816
[160]	valid_0's rmse: 0.876703
[170]	valid_0's rmse: 0.875295
[180]	valid_0's rmse: 0.873887
[190]	valid_0's rmse: 0.872521
[200]	valid_0's rmse: 0.871154
[210]	valid_0's rmse: 0.869823
[220]	valid_0's rmse: 0.868482
[230]	valid_0's rmse: 0.867166
[240]	valid_0's rmse: 0.865872
[250]	valid_0's rmse: 0.864607
[260]	valid_0's rmse: 0.863334
[270]	valid_0's rmse: 0.862117
[280]	valid_0's rmse: 0.860869
[290]	valid_0's rmse: 0.859629
[300]	valid_0's rmse: 0.858423
[310]	valid_0's rmse

[2600]	valid_0's rmse: 0.777565
[2610]	valid_0's rmse: 0.777435
[2620]	valid_0's rmse: 0.777323
[2630]	valid_0's rmse: 0.777199
[2640]	valid_0's rmse: 0.777106
[2650]	valid_0's rmse: 0.776995
[2660]	valid_0's rmse: 0.776876
[2670]	valid_0's rmse: 0.776831
[2680]	valid_0's rmse: 0.776716
[2690]	valid_0's rmse: 0.776609
[2700]	valid_0's rmse: 0.776497
[2710]	valid_0's rmse: 0.776411
[2720]	valid_0's rmse: 0.776331
[2730]	valid_0's rmse: 0.776217
[2740]	valid_0's rmse: 0.776096
[2750]	valid_0's rmse: 0.775994
[2760]	valid_0's rmse: 0.775902
[2770]	valid_0's rmse: 0.775794
[2780]	valid_0's rmse: 0.77575
[2790]	valid_0's rmse: 0.775642
[2800]	valid_0's rmse: 0.77558
[2810]	valid_0's rmse: 0.775519
[2820]	valid_0's rmse: 0.775566
[2830]	valid_0's rmse: 0.775547
[2840]	valid_0's rmse: 0.775493
[2850]	valid_0's rmse: 0.775443
[2860]	valid_0's rmse: 0.775468
[2870]	valid_0's rmse: 0.775472
[2880]	valid_0's rmse: 0.775527
[2890]	valid_0's rmse: 0.775497
[2900]	valid_0's rmse: 0.775519
[2910]	val

In [22]:
pickle.dump(model_lgb, open( "pickled/model_lgb", "wb"), protocol=4)

#model_lgb = pickle.load( open( "pickled/model_lgb", "rb" ) )

In [47]:
pd.set_option('display.max_columns', None)  
pd.set_option('display.expand_frame_repr', False)
pd.set_option('max_colwidth', -1)
#x_train[cb_features].sample(10)
training.dtypes

item_id                                     int16  
shop_id                                     uint8  
date_block_num                              uint8  
item_category_id                            uint8  
month                                       uint8  
year                                        uint16 
item_first_block                            uint8  
item_last_block                             uint8  
is_first_two_blocks                         object 
is_last_two_blocks                          object 
item_units                                  float32
item_mean_units_block                       float32
item_day_units                              int16  
item_mean_units_day                         float32
item_max_units_block                        int16  
item_min_units_block                        int16  
item_max_units_day                          int16  
item_min_units_day                          int8   
item_turnover                               int32  
item_mean_tu

In [39]:
test = test.set_index('item_id').join(items.set_index('item_id'))
test['date_block_num'] = 34

In [36]:
transactions_items = pd.read_pickle("pickled/transactions_items")
transactions_items_dtypes = transactions_items.dtypes
test = pd.merge(test, transactions_items[transactions_items_columns].drop_duplicates('item_id'), on=['item_id'], how='left', copy=False)

del transactions_items
fillnas(test, transactions_items_columns, transactions_items_dtypes)
downcast(test, transactions_items_columns, transactions_items_dtypes)
gc.collect()

28

In [38]:
test.head()

,ID,shop_id,item_id,item_first_block,item_last_block,is_first_two_blocks,is_last_two_blocks,item_units,item_mean_units_block,item_day_units,...,item_price_amplitude,item_deviation_mean_category_price,item_first_two_blocks_units,item_last_two_blocks_units,item_fluctuation_units_first_last_blocks,item_first_two_blocks_mean_price,item_last_two_blocks_mean_price,item_fluctuation_price_first_last_blocks,item_share_of_total_units,item_share_of_total_turnover
0,0,5,5037,20,33,False,False,1270.0,126.292007,5,...,246.996002,27.636459,127.0,135.0,6.299212,2570.850342,1077.921509,-58.071400,0.060897,0.110578
1,1,5,5320,0,0,NaN,NaN,0.0,0.000000,0,...,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,2,5,5233,27,33,False,False,507.0,99.613335,1,...,100.166946,-46.963413,190.0,122.0,-35.789474,703.523010,901.857117,28.191566,0.024311,0.018165
3,3,5,5232,31,33,False,False,141.0,50.446156,7,...,100.166946,-45.054775,76.0,76.0,-0.000000,895.618408,895.618408,-0.000000,0.006761,0.004957
4,4,5,5268,0,0,NaN,NaN,0.0,0.000000,0,...,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [40]:
transactions_items_blocks = pd.read_pickle("pickled/transactions_items_blocks")
transactions_items_blocks_dtypes = transactions_items_blocks.dtypes

test = pd.merge(test, transactions_items_blocks[transactions_items_blocks_columns]\
                    .drop_duplicates(['item_id', 'date_block_num']), on=['item_id','date_block_num'], how='left', copy=False)

del transactions_items_blocks
test.fillna(0, inplace=True)
downcast(test, transactions_items_blocks_columns, transactions_items_blocks_dtypes)
gc.collect()

11238

In [41]:
transactions_categories = pd.read_pickle("pickled/transactions_categories")
transactions_categories_dtypes = transactions_categories.dtypes
test = pd.merge(test, transactions_categories[transactions_categories_columns]\
                    .drop_duplicates('item_category_id'), on=['item_category_id'], how='left', copy=False)

del transactions_categories
fillnas(test, transactions_categories_columns, transactions_categories_dtypes)
downcast(test, transactions_categories_columns, transactions_categories_dtypes)
gc.collect()

14

In [42]:
transactions_categories_blocks = pd.read_pickle("pickled/transactions_categories_blocks")
transactions_categories_blocks_dtypes = transactions_categories_blocks.dtypes
test = pd.merge(test, transactions_categories_blocks[transactions_categories_blocks_columns]\
                    .drop_duplicates(['item_category_id', 'date_block_num']), on=['item_category_id', 'date_block_num'], how='left', copy=False)

del transactions_categories_blocks
fillnas(test, transactions_categories_blocks_columns, transactions_categories_blocks_dtypes)
downcast(test, transactions_categories_blocks_columns, transactions_categories_blocks_dtypes)
gc.collect()

14

In [43]:
transactions_shops = pd.read_pickle("pickled/transactions_shops")
transactions_shops_dtypes = transactions_shops.dtypes
test = pd.merge(test, transactions_shops[transactions_shops_columns]\
                    .drop_duplicates('shop_id'), on=['shop_id'], how='left', copy=False)

del transactions_shops
fillnas(test, transactions_shops_columns, transactions_shops_dtypes)
downcast(test, transactions_shops_columns, transactions_shops_dtypes)
gc.collect()

14

In [45]:
transactions_shops_blocks = pd.read_pickle("pickled/transactions_shops_blocks")
transactions_shops_blocks_dtypes = transactions_shops_blocks.dtypes
test = pd.merge(test, transactions_shops_blocks[transactions_shops_blocks_columns]\
                    .drop_duplicates(['shop_id', 'date_block_num']), on=['shop_id', 'date_block_num'], how='left', copy=False)

del transactions_shops_blocks
fillnas(test, transactions_shops_blocks_columns, transactions_shops_blocks_dtypes)
downcast(test, transactions_shops_blocks_columns, transactions_shops_blocks_dtypes)
gc.collect()

14

In [46]:
transactions_shops_categories = pd.read_pickle("pickled/transactions_shops_categories")
transactions_shops_categories_dtypes = transactions_shops_categories.dtypes
test = pd.merge(test, transactions_shops_categories[transactions_shops_categories_columns]\
                    .drop_duplicates(['shop_id','item_category_id']), on=['shop_id','item_category_id'], how='left', copy=False)

del transactions_shops_categories
fillnas(test, transactions_shops_categories_columns, transactions_shops_categories_dtypes)
downcast(test, transactions_shops_categories_columns, transactions_shops_categories_dtypes)
gc.collect()

14

In [44]:
transactions_shops_categories_blocks = pd.read_pickle("pickled/transactions_shops_categories_blocks")
transactions_shops_categories_blocks_dtypes = transactions_shops_categories_blocks.dtypes
test = pd.merge(test, transactions_shops_categories_blocks[transactions_shops_categories_blocks_columns]\
                    .drop_duplicates(['shop_id','item_category_id','date_block_num']),\
                    on=['shop_id','item_category_id','date_block_num'], how='left', copy=False)

del transactions_shops_categories_blocks
fillnas(test, transactions_shops_categories_blocks_columns, transactions_shops_categories_blocks_dtypes)
downcast(test, transactions_shops_categories_blocks_columns, transactions_shops_categories_blocks_dtypes)
gc.collect()

14

In [49]:
gc.collect()
def lagged_name(lag_column, lag):
    return "%s_lag_%d" % (lag_column, lag)

lag_columns = [
 'item_block_units',
 'item_block_turnover',
 'item_mean_price_block',
 ]

idx_columns = ['item_id']

test = add_lag_features(test,lag_columns,idx_columns)

1
2
3


In [51]:
gc.collect()
def lagged_name(lag_column, lag):
    return "%s_lag_%d" % (lag_column, lag)

lag_columns = [
 'category_block_units',
 'category_block_turnover',
 'category_mean_price_block',
 'subcategory_block_units',
 'subcategory_block_turnover',
 'subcategory_mean_price_block',
 ]

idx_columns = ['item_category_id']

test = add_lag_features(test,lag_columns,idx_columns)

1
2
3


In [52]:
gc.collect()
def lagged_name(lag_column, lag):
    return "%s_lag_%d" % (lag_column, lag)

lag_columns = [
 'shop_block_units',
 'shop_block_turnover',
 'shop_mean_price_block',
 'area_block_units',
 'area_block_turnover',
 'area_mean_price_block'
 ]

idx_columns = ['shop_id']


test = add_lag_features(test,lag_columns,idx_columns)

1
2
3


In [3]:
#gc.collect()
#training.to_pickle("pickled/training_mid_lags")
#training = pd.read_pickle("pickled/training_mid_lags")

In [53]:
lag_columns = [
  'shop_category_block_units',
 'shop_category_turnover',
 'shop_category_mean_price_block'
 ]

idx_columns = ['shop_id','item_category_id']


test = add_lag_features(test,lag_columns,idx_columns)

1
2
3


In [54]:
test.drop(columns=['lagged_block'],inplace=True)

In [60]:
for i in range(1,13):
    test[str(i)] = False

test['11'] = True


In [83]:
cb_preds = cb_model.predict(test[cb_features])
cb_preds.clip(0,20,out=cb_preds)

array([0.3875993 , 0.31189891, 0.50810608, ..., 0.48477869, 0.48491959,
       0.47320142])

In [73]:
lg_preds = model_lgb.predict(test[lg_features])
lg_preds.clip(0,20,out=lg_preds)

array([0.33054925, 0.13510253, 0.42999917, ..., 0.40670206, 0.39667443,
       0.38160782])

In [74]:

xg_preds = xg_model.predict(xgb.DMatrix(test[xg_features]))
xg_preds.clip(0,20,out=xg_preds)

array([0.37465316, 0.21515614, 0.47788906, ..., 0.4456131 , 0.40560868,
       0.45044884], dtype=float32)

In [81]:
preds = np.mean(np.array([cb_preds,lg_preds,xg_preds]),axis=0)

In [84]:
submission = test.loc[:,['ID']]
submission['item_cnt_month'] = cb_preds.astype(int)
#submission['item_cnt_month'] = ensemble_preds.astype(int)


submission.to_csv('submission.csv', index=False)